<a href="https://colab.research.google.com/github/globalenglish01/Billion/blob/main/parse_chatgpt_html.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install beautifulsoup4 pandas openpyxl

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import re
import pandas as pd
from bs4 import BeautifulSoup

def clean_pre_text(text):
    """
    清理 <pre> 标签内的多余文字，例如开头的“plaintext”、“复制代码”等。
    """
    remove_phrases = ['plaintext', '复制代码', 'コピーコード']
    for phrase in remove_phrases:
        text = text.replace(phrase, '')
    # 去除开头结尾多余空白符
    return text.lstrip('\n\r ').rstrip()

# 1. 读取文件，忽略非法字符
def read_html_file(filepath):
    with open(filepath, 'rb') as f:
        data = f.read()
    return data.decode('utf-8', errors='ignore')

# 2. 解析内容，提取每页三部分
def parse_pages(html_text):
    soup = BeautifulSoup(html_text, 'html.parser')

    # 获取所有文本块（包含标题和内容）
    # 观察你提供的html结构，大致是 <p>标题</p> + <pre>内容</pre>，循环顺序依次出现
    # 通过循环所有<p>标签，判断标题，配合后面的<pre>拿内容
    ps = soup.find_all('p')
    pres = soup.find_all('pre')

    pages = []
    # 这里假设标题和内容是交替出现的，即p - pre - p - pre - p - pre这种结构
    # 或者三个一组 p-pre-p-pre-p-pre依次对应一页3部分
    # 通过遍历p和pre，收集"第X页 原文"、"第X页 中文翻译"、"第X页 逐行详细解析"
    # 并将内容保存

    # 建立标题和内容的对应关系，假设p和pre对应序列是严格的
    p_texts = [p.get_text(strip=True) for p in ps]
    pre_texts_raw = [pre.get_text() for pre in pres]
    pre_texts = [clean_pre_text(t) for t in pre_texts_raw]

    # 存储临时信息
    page_num = None
    part_dict = {}  # keys: 原文, 中文翻译, 逐行详细解析
    results = []

    # 标题关键词匹配
    def get_part_type(text):
        if '原文' in text:
            return '原文'
        elif '中文翻译' in text:
            return '中文翻译'
        elif '逐行詳細解析' in text or '逐行详细解析' in text:
            return '逐行详细解析'
        return None

    i_p = 0  # p索引
    i_pre = 0  # pre索引

    while i_p < len(p_texts) and i_pre < len(pre_texts):
        title = p_texts[i_p]
        content = pre_texts[i_pre]

        # 匹配页码
        page_match = re.search(r'第(\d+)页', title)
        part = get_part_type(title)

        if page_match and part:
            current_page = int(page_match.group(1))
            # 如果是新的一页，先保存上一个
            if page_num is not None and current_page != page_num:
                # 确保三个部分都存在，不存在则设空字符串
                results.append({
                    '页码': "第" + str(page_num) + "页",
                    '原文': part_dict.get('原文', ''),
                    '中文翻译': part_dict.get('中文翻译', ''),
                    '逐行详细解析': part_dict.get('逐行详细解析', ''),
                })
                part_dict = {}

            page_num = current_page
            part_dict[part] = content

            i_p += 1
            i_pre += 1
        else:
            # 遇到不匹配标题，跳过，防止死循环
            i_p += 1
            i_pre += 1

    # 保存最后一页
    if page_num is not None:
        results.append({
            '页码': "第" + str(page_num) + "页",
            '原文': part_dict.get('原文', ''),
            '中文翻译': part_dict.get('中文翻译', ''),
            '逐行详细解析': part_dict.get('逐行详细解析', ''),
        })

    return results

# 3. 保存为Excel
def save_to_excel(data_list, excel_path):
    df = pd.DataFrame(data_list)
    df = df.sort_values(by='页码')
    df.to_excel(excel_path, index=False)

# 4. 主函数示例
def main():
    #filepath = '/content/drive/MyDrive/chatgpt_export0.html'  # 替换成你的路径
    filepath = '/content/drive/MyDrive/chatgpt_export.html'  # 替换成你的路径
    excel_path = '/content/drive/MyDrive/parsed_pages.xlsx'  # 输出路径

    html_text = read_html_file(filepath)
    parsed_pages = parse_pages(html_text)
    save_to_excel(parsed_pages, excel_path)
    print(f'解析完成，已保存至 {excel_path}')

# 运行主函数
if __name__ == '__main__':
    main()

解析完成，已保存至 /content/drive/MyDrive/parsed_pages.xlsx


In [ ]:

from bs4 import BeautifulSoup
import re
import pandas as pd

def clean_pre_text(text):
    remove_phrases = ['plaintext', '复制代码', 'コピーコード']
    for phrase in remove_phrases:
        text = text.replace(phrase, '')
    return text.strip()

def get_part_type(text):
    if '原文' in text:
        return '原文'
    elif '中文翻译' in text:
        return '中文翻译'
    elif '逐行詳細解析' in text or '逐行详细解析' in text:
        return '逐行详细解析'
    return None

def parse_html_by_positions(html_text):
    soup = BeautifulSoup(html_text, 'html.parser')

    ps = soup.find_all('p')
    pres = soup.find_all('pre')

    # 收集标题：页码、部分类型、p标签位置
    titles = []
    for i, p_tag in enumerate(ps):
        text = p_tag.get_text(strip=True)
        m = re.search(r'第(\d+)页', text)
        part = get_part_type(text)
        if m and part:
            page_num = int(m.group(1))
            titles.append({'page': page_num, 'part': part, 'p_index': i})

    # 所有pre标签的位置
    pre_positions = list(range(len(pres)))

    # 记录结果：页码映射到部分内容字典
    pages = {}

    # 用于找到每个标题对应的pre标签
    p_to_pre_map = {}

    # 遍历每个标题，找第一个pre标签的位置比p标签位置大的那个
    # 这里用ps和pres不在同一序列，这里用p标签序号对应文档中的位置不好用，改用标签在文档中的索引即可
    # 方案：先找p标签在文档中的sourceline，然后找pre标签的sourceline，匹配

    # 取p和pre标签在文档中的sourceline(行号)（有时解析器可用）
    # 这里用bs4默认不能保证sourceline存在，改用在DOM中索引顺序来判断：

    # ps[i] 和 pres[j] 在文档中的顺序：
    # 我们认为p和pre交替出现，p[i].index < pre[j].index < p[i+1].index

    # 所以先找所有p和pre的标签在文档的position序号
    # bs4没有直接提供DOM中的绝对位置索引，换思路：html文档中p标签和pre标签的顺序决定他们的出现顺序

    # 这里先把所有p和pre合并排序
    all_tags = []
    for tag in ps:
        all_tags.append(('p', tag))
    for tag in pres:
        all_tags.append(('pre', tag))

    # 在html中出现的顺序
    all_tags_sorted = sorted(all_tags, key=lambda x: x[1].sourceline if x[1].sourceline else 0)

    # 由于sourceline不一定存在，改用顺序索引来模拟
    # 先构造一个列表，p和pre标签按照在html文本中的顺序排序，sourceline无效时使用下面代码

    # 但如果sourceline不可用，不能用此方法。改用原始HTML文本用正则提取页码和内容匹配
    # 因为bs4对sourceline支持不完全

    # 换用正则和HTML文本解析更准确方案：
    return None

def parse_html_with_regex(html_text):
    """
    用正则匹配页码标题和后面对应内容的pre，适合复杂页面。
    """

    # 匹配页码和部分标题，如 “第123页 原文” ，后面跟着 <pre>...</pre>
    pattern = re.compile(
        r'<p[^>]*?>\s*(第(\d+)页[^<]*)\s*</p>\s*'    # <p>标题
        r'<pre[^>]*?>\s*(.*?)\s*</pre>',              # <pre>内容
        re.DOTALL
    )

    matches = pattern.findall(html_text)

    pages = {}
    for full_title, page_str, content in matches:
        part = None
        if '原文' in full_title:
            part = '原文'
        elif '中文翻译' in full_title:
            part = '中文翻译'
        elif '逐行詳細解析' in full_title or '逐行详细解析' in full_title:
            part = '逐行详细解析'
        else:
            continue  # 跳过无法识别的

        page_num = int(page_str)
        if page_num not in pages:
            pages[page_num] = {}
        # 清理内容
        content_clean = clean_pre_text(content)
        pages[page_num][part] = content_clean

    # 转换为列表并补全三列
    results = []
    for pnum in sorted(pages.keys()):
        page_data = pages[pnum]
        results.append({
            '页码': pnum,
            '原文': page_data.get('原文', ''),
            '中文翻译': page_data.get('中文翻译', ''),
            '逐行详细解析': page_data.get('逐行详细解析', ''),
        })
    return results

# 使用示例
#html_file_path = '/content/your_file.html'
html_file_path = '/content/drive/MyDrive/chatgpt_export0.html'  # 替换成你的路径
excel_path = '/content/drive/MyDrive/parsed_pages.xlsx'  # 输出路径
with open(html_file_path, 'rb') as f:
    html_content = f.read().decode('utf-8', errors='ignore')

pages_data = parse_html_with_regex(html_content)

import pandas as pd
df = pd.DataFrame(pages_data, columns=['页码', '原文', '中文翻译', '逐行详细解析'])
df.to_excel(excel_path, index=False)
print('解析完成，保存为 parsed_pages.xlsx')

解析完成，保存为 parsed_pages.xlsx


In [ ]:
# 1. 挂载Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 2. 导入所需库
import re
from bs4 import BeautifulSoup
import pandas as pd

# 3. 解析纯文本的辅助函数
def clean_html_text(html_fragment):
    """用BeautifulSoup去除HTML标签，得到纯文本，并去除“plaintext”、“复制代码”等无意义文字"""
    text = BeautifulSoup(html_fragment, 'html.parser').get_text()
    # 去掉“plaintext”、“复制代码”等关键词，及多余空行
    for noise in ['plaintext', '复制代码', 'コピーコード', 'copy code']:
        text = text.replace(noise, '')
    text = '\n'.join([line.strip() for line in text.splitlines() if line.strip() != ''])
    return text

# 4. 解析函数：提取每页的三部分内容
def parse_html_pages(html_text):
    # 正则匹配所有 <p>标题 + <pre>内容，标题格式为第XXX页 XX，内容是<pre>里的内容
    pattern = re.compile(
        r'<p[^>]*?>\s*(第(\d+)页[^<]*)\s*</p>\s*'    # 匹配标题：例如“第378页 原文”
        r'<pre[^>]*?>\s*(.*?)\s*</pre>',              # 匹配紧跟的pre内容
        re.DOTALL
    )
    matches = pattern.findall(html_text)

    pages = {}
    for full_title, page_str, content in matches:
        # 判断是哪部分内容
        if '原文' in full_title:
            part = '原文'
        elif '中文翻译' in full_title:
            part = '中文翻译'
        elif '逐行詳細解析' in full_title or '逐行详细解析' in full_title:
            part = '逐行详细解析'
        else:
            # 避免无关段落干扰
            continue

        page_num = int(page_str)
        if page_num not in pages:
            pages[page_num] = {}

        clean_text = clean_html_text(content)
        pages[page_num][part] = clean_text

    # 组装成列表，方便转DataFrame
    results = []
    for page_num in sorted(pages.keys()):
        page_data = pages[page_num]
        results.append({
            '页码': page_num,
            '原文': page_data.get('原文', ''),
            '中文翻译': page_data.get('中文翻译', ''),
            '逐行详细解析': page_data.get('逐行详细解析', ''),
        })
    return results
import pandas as pd

def export_for_anki(page_contents, output_path):
    """
    page_contents: 解析出来的结构，如 [{'page': 378, 'original': '...', 'translation': '...', 'analysis': '...'}, ...]
    output_path: 导出路径，比如 '/content/anki_cards.tsv'
    """
    anki_rows = []

    for page in page_contents:
        front = page.get('original', '').strip()
        back_parts = []

        if page.get('translation'):
            back_parts.append("<b>中文翻译：</b><br>" + page['translation'].replace('\n', '<br>'))
        if page.get('analysis'):
            back_parts.append("<b>逐行解析：</b><br>" + page['analysis'].replace('\n', '<br>'))

        back = "<br><br>".join(back_parts)
        anki_rows.append([front, back])

    # 转为 DataFrame
    df = pd.DataFrame(anki_rows, columns=['Front', 'Back'])

    # 保存为制表符分隔的 UTF-8 文件
    df.to_csv(output_path, sep='\t', index=False, header=False, encoding='utf-8')
    print(f"✅ Anki 卡片文件已导出到 {output_path}")

import pandas as pd

def export_with_section(page_contents, output_path):
    """
    page_contents: [{'page': 378, 'original': '...', 'translation': '...', 'analysis': '...'}, ...]
    output_path: 导出 CSV 路径
    """
    rows = []
    for page in page_contents:
        rows.append([
            page.get('page', ''),   # Page
            '',                     # Section (空)
            page.get('original', '').strip(),     # Original
            page.get('translation', '').strip(),  # Translation
            page.get('analysis', '').strip()      # Analysis
        ])

    df = pd.DataFrame(rows, columns=['Page', 'Section', 'Original', 'Translation', 'Analysis'])
    df.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"✅ 已导出 CSV 文件: {output_path}")



# 5. 读取你的HTML文件路径（请替换成你自己的文件路径）
#html_file_path = '/content/drive/MyDrive/chatgpt_export.html'
html_file_path = '/content/drive/MyDrive/chatgpt_export0.html'


# 用二进制模式读取，忽略无法解码的字节
with open(html_file_path, 'rb') as f:
    html_content = f.read().decode('utf-8', errors='ignore')

# 6. 调用解析函数
page_contents = parse_html_pages(html_content)

# 7. 转成DataFrame并导出Excel
df = pd.DataFrame(page_contents, columns=['页码', '原文', '中文翻译', '逐行详细解析'])
output_path = '/content/drive/MyDrive/parsed_pages.xlsx'
df.to_excel(output_path, index=False)

print(f"完成！Excel文件已保存到：{output_path}")

# 使用示例（假设 page_contents 已经是你解析后的结构）
#Anki_output_path = '/content/drive/MyDrive/aws_anki_cards.tsv'
#export_for_anki(page_contents, Anki_output_path)

# 调用
#export_with_section(page_contents, '/content/drive/MyDrive/aws_anki_csv.csv')
# 在 'Page' 和 'Original' 之间插入一个空列 Section
df.insert(1, 'Section', '')

# 导出 CSV
csv_output_path = '/content/drive/MyDrive/aws_anki_csv.csv'
df.to_csv(csv_output_path, index=False, encoding='utf-8-sig')
print(f"完成！CSV文件已保存到：{csv_output_path}")

print(df)

# 8. （可选）展示部分内容确认

for page in page_contents[:3]:
    print(f"第{page['页码']}页 原文片段:\n{page['原文'][:300]}\n")
    print(f"第{page['页码']}页 中文翻译片段:\n{page['中文翻译'][:300]}\n")
    print(f"第{page['页码']}页 逐行详细解析片段:\n{page['逐行详细解析'][:300]}\n")
    print("-" * 40)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
完成！Excel文件已保存到：/content/drive/MyDrive/parsed_pages.xlsx
完成！CSV文件已保存到：/content/drive/MyDrive/aws_anki_csv.csv
    页码 Section 原文 中文翻译                                             逐行详细解析
0  310                  ⓫【原文】（1つのSigned URL で1つのファイルにアクセス可能）\n【発音】（ひとつ...
第310页 原文片段:


第310页 中文翻译片段:


第310页 逐行详细解析片段:
⓫【原文】（1つのSigned URL で1つのファイルにアクセス可能）
【発音】（ひとつ の サインド ユーアールエル で ひとつ の ファイル に アクセス かのう）
【翻訳】（一个签名 URL 可访问一个文件）
【単語】
① 1つ（ひとつ）：一个
② Signed URL（サインド ユーアールエル）：签名 URL
③ で：表示动作发生的场所或手段的助词
④ ファイル（ファイル）：文件
⑤ に：表示动作的目标的助词
⑥ アクセス（アクセス）：访问
⑦ 可能（かのう）：可能
【語法】
「～で」这里表示通过某手段（签名 URL）进行访问。
「～にアクセス可能」表示可以访问某对象。
❿【原文】S

----------------------------------------


In [ ]:
# 运行前安装依赖（Colab可运行）
#!pip install beautifulsoup4 lxml pandas openpyxl

# 1. 挂载Google Drive
from google.colab import drive
drive.mount('/content/drive')

import re
import pandas as pd
from bs4 import BeautifulSoup

# -----------------------
# 配置：文件路径（修改为你的路径）
# -----------------------
HTML_FILE_PATH = '/content/drive/MyDrive/chatgpt_export0.html'  # <- 修改
OUTPUT_MID_EXCEL = '/content/drive/MyDrive/codeblocks_with_meta.xlsx'   # 中间表：代码块、页码、类别、其他
OUTPUT_FINAL_EXCEL = '/content/drive/MyDrive/final_original_translation_analysis.xlsx'  # 最终三列表

# -----------------------
# 读取 HTML（以二进制读，utf-8 解码并忽略非法字节）
# -----------------------
with open(HTML_FILE_PATH, 'rb') as f:
    html_bytes = f.read()
html_text = html_bytes.decode('utf-8', errors='ignore')

soup = BeautifulSoup(html_text, 'lxml')  # 使用 lxml 更稳健

# -----------------------
# 提取页面中按顺序出现的“代码块”
# 优先采集 <pre>，若 <pre> 不够，再取 <code> 或含language-plaintext类的
# 保持文档顺序：BS4 find_all 返回的顺序即文档顺序
# -----------------------
code_tags = []
# 首先收集所有 <pre> 标签
for tag in soup.find_all(['pre', 'code']):
    # 过滤空的code/pre
    txt = tag.get_text()
    if txt and txt.strip():
        code_tags.append((tag.name, txt))

# 如果没有任何 pre/code，再尝试找带特定 class 的容器（fallback）
if not code_tags:
    for div in soup.find_all('div'):
        txt = div.get_text()
        if txt and txt.strip():
            code_tags.append(('div', txt))

# 初始 DataFrame 列：代码块, 页码, 类别, 其他
df_mid = pd.DataFrame(columns=['代码块', '页码', '类别', '其他'])

# 将提取到的代码块按顺序放入中间表（页码/类别/其他初始为空）
for txt in [t[1] for t in code_tags]:
    df_mid = df_mid.append({'代码块': txt, '页码': '', '类别': '', '其他': ''}, ignore_index=True)

# -----------------------
# 寻找并处理代码块内的“第XX页”页码行，按要求拆分
# 规则：
# - 对每个代码块，按行检测去掉空格后开头为第\d+页 或 第\d+頁 的行
# - 当找到这样的行（可能多个），将该代码块拆分：
#     - 如果 header 在块内，则把 header 以及 header 后的内容作为一个新块（header行不包含在最终代码块内容--但将 header 文本放入 "其他" 字段）
#     - 对于 header 捕获的页码 xx：将该 header 行后紧接着的下一个代码块（在中间表顺序上的下一个） 的 '页码' 设置为 xx，并把 header 行内容写入该行的 '其他' 字段
# - 若 header 出现在块中间/末尾：把 header 及其后内容分割成新的记录，原记录用前半部分替代
#
# 说明：为简化实现并保证稳定性，按如下实现：
# - 找到 header 行在该代码块行内的位置
# - 以 header 行作为“划分点”：把原块拆为多段（段不包含 header 行）
# - header 所指页码页号将被赋给“紧随 header 的下一个代码块”（若存在）——这里我们把 header 所指页码赋给拆分结果中对应的“段”的页码字段，同样把 header 行文本放入该段的“其他”字段
# -----------------------

# 正则匹配 "第123页" 或 "第123頁"，允许中间有空白或非数字的情况也尽量捕获数字
page_header_re = re.compile(r'^\s*第\s*([0-9０-９]+)\s*[页頁]\s*', re.UNICODE)

def normalize_number_str(num_str):
    # 将全角数字转换为半角
    trans = str.maketrans('０１２３４５６７８９', '0123456789')
    return num_str.translate(trans)

new_rows = []
for idx, row in df_mid.iterrows():
    block = row['代码块']
    lines = block.splitlines()
    header_positions = []  # list of (line_index, matched_number_str, original_line_text)
    for i, ln in enumerate(lines):
        # 移除所有空白判断是否以 第xx页 开头
        compact = re.sub(r'\s+', '', ln)
        m = page_header_re.match(compact)
        if m:
            num_str = normalize_number_str(m.group(1))
            header_positions.append((i, num_str, ln.strip()))
    if not header_positions:
        # 无拆分，直接保留
        new_rows.append({'代码块': block, '页码': row['页码'], '类别': row['类别'], '其他': row['其他']})
    else:
        # 有拆分点。我们把原块拆成若干片段：
        splits = []
        prev = 0
        for pos, num_str, header_line in header_positions:
            # part before header (保留)
            if pos > prev:
                part_before = '\n'.join(lines[prev:pos]).strip()
                if part_before:
                    splits.append({'text': part_before, 'is_header_bound': False, 'page_from_header': None, 'header_line': ''})
            # header-bound part: collect following lines until next header or EOF
            # find start index after header line
            next_start = pos + 1
            # find next header position index
            # (we will assign content after header to page num)
            # to be decided after we determine next header index
            # we'll just mark header here; content will be gathered in next loop
            prev = next_start
        # after processing headers, add tail
        if prev < len(lines):
            tail = '\n'.join(lines[prev:]).strip()
            if tail:
                splits.append({'text': tail, 'is_header_bound': False, 'page_from_header': None, 'header_line': ''})

        # The above split logic is conservative; instead implement direct linear processing:
        # We'll iterate lines and whenever we encounter header, start a new segment for content after header, and attach header info to that segment.

        segments = []
        current_seg_lines = []
        pending_header = None  # tuple (num_str, header_text)
        i = 0
        while i < len(lines):
            compact = re.sub(r'\s+', '', lines[i])
            m = page_header_re.match(compact)
            if m:
                # when see header, if current_seg_lines not empty, finalize it as a standalone segment
                if current_seg_lines:
                    segments.append({'text': '\n'.join(current_seg_lines).strip(), 'header_num': None, 'header_line': ''})
                    current_seg_lines = []
                # set pending header: the content after this header will be a segment marked with this header
                num = normalize_number_str(m.group(1))
                header_txt = lines[i].strip()
                # start new current_seg_lines for content after header
                current_seg_lines = []
                # set pending header to be applied to next created segment
                pending_header = (num, header_txt)
                i += 1
                # collect following lines until next header occurrence
                while i < len(lines):
                    compact2 = re.sub(r'\s+', '', lines[i])
                    m2 = page_header_re.match(compact2)
                    if m2:
                        break
                    current_seg_lines.append(lines[i])
                    i += 1
                # finalize this header-bound segment
                seg_text = '\n'.join(current_seg_lines).strip()
                segments.append({'text': seg_text, 'header_num': pending_header[0], 'header_line': pending_header[1]})
                pending_header = None
                current_seg_lines = []
            else:
                current_seg_lines.append(lines[i])
                i += 1
        # if leftover
        if current_seg_lines:
            seg_text = '\n'.join(current_seg_lines).strip()
            if seg_text:
                segments.append({'text': seg_text, 'header_num': None, 'header_line': ''})

        # Now segments is a list; replace original row with these segments in place
        for seg in segments:
            # Note: if seg has header_num, set that as 页码 and put header_line in 其他
            if seg['header_num']:
                new_rows.append({'代码块': seg['text'], '页码': seg['header_num'], '类别': '', '其他': seg['header_line']})
            else:
                new_rows.append({'代码块': seg['text'], '页码': '', '类别': '', '其他': ''})

# 重建中间表
df_mid = pd.DataFrame(new_rows, columns=['代码块', '页码', '类别', '其他'])

# -------

Mounted at /content/drive


AttributeError: 'DataFrame' object has no attribute 'append'

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import re

# 1. 读取 HTML
#html_file_path = "/content/chatgpt_session.html"  # 你的HTML文件路径
html_file_path = '/content/drive/MyDrive/chatgpt_export0.html'  # <- 修改
OUTPUT_MID_EXCEL = '/content/drive/MyDrive/codeblocks_with_meta.xlsx'   # 中间表：代码块、页码、类别、其他
OUTPUT_FINAL_EXCEL = '/content/drive/MyDrive/final_original_translation_analysis.xlsx'  # 最终三列表
with open(html_file_path, "r", encoding="utf-8", errors="ignore") as f:
    html_content = f.read()

# 2. 用 BeautifulSoup 解析 HTML
soup = BeautifulSoup(html_content, "html.parser")

# 3. 提取所有代码块（<code>标签）
code_tags = []
for code in soup.find_all("code"):
    code_text = code.get_text("\n")  # 取出代码块内容
    if code_text.strip():
        code_tags.append(code_text)

# 4. 初始化 DataFrame
df_mid = pd.DataFrame(columns=["代码块", "页码", "类别", "其他"])

# 5. 用 concat 一次性构造
df_mid = pd.concat([
    pd.DataFrame({"代码块": [txt], "页码": [""], "类别": [""], "其他": [""]})
    for txt in code_tags
], ignore_index=True)

# 6. 拆分含“第xx页”标记的行
pattern_page = re.compile(r"第(\d+)页")
new_rows = []

for _, row in df_mid.iterrows():
    text = row["代码块"]
    lines = text.splitlines()
    split_indexes = []

    for i, line in enumerate(lines):
        if pattern_page.search(line.strip().replace(" ", "")):
            split_indexes.append(i)

    if split_indexes:
        start_idx = 0
        for idx in split_indexes:
            if idx > start_idx:
                # 保存之前的部分
                new_rows.append([ "\n".join(lines[start_idx:idx]), row["页码"], row["类别"], row["其他"] ])
            # 保存页码信息
            page_match = pattern_page.search(lines[idx].replace(" ", ""))
            if page_match:
                page_num = page_match.group(1)
                new_rows.append(["", page_num, "", lines[idx]])  # 页码在“页码”列，原始行放“其他”列
            start_idx = idx + 1
        # 保存最后一部分
        if start_idx < len(lines):
            new_rows.append(["\n".join(lines[start_idx:]), row["页码"], row["类别"], row["其他"]])
    else:
        new_rows.append([text, row["页码"], row["类别"], row["其他"]])

df_mid = pd.DataFrame(new_rows, columns=["代码块", "页码", "类别", "其他"])

# 7. 分类
def contains_japanese(text):
    return bool(re.search(r"[\u3040-\u30ff\u4e00-\u9fff]", text))

def contains_chinese(text):
    return bool(re.search(r"[\u4e00-\u9fff]", text))

for i, row in df_mid.iterrows():
    if "原文" in str(row["其他"]):
        df_mid.at[i, "类别"] = "原文"
    elif "翻译" in str(row["其他"]):
        df_mid.at[i, "类别"] = "中文翻译"
    elif "逐行详细解析" in str(row["其他"]):
        df_mid.at[i, "类别"] = "逐行解析"
    else:
        code_text = str(row["代码块"])
        if contains_chinese(code_text) and contains_japanese(code_text):
            df_mid.at[i, "类别"] = "逐行解析"
        elif not contains_japanese(code_text) and contains_chinese(code_text):
            df_mid.at[i, "类别"] = "中文翻译"
        else:
            df_mid.at[i, "类别"] = "原文"

# 8. 导出中间Excel
df_mid.to_excel(OUTPUT_MID_EXCEL, index=False)
print("已保存中间表 mid_result.xlsx")

已保存中间表 mid_result.xlsx


In [ ]:
# 运行前请确保安装依赖（在 Colab 可运行以下命令）
# !pip install beautifulsoup4 lxml pandas openpyxl

import re
import pandas as pd
from bs4 import BeautifulSoup

# ---------------------------
# 配置：将下面路径替换为你的文件路径
# ---------------------------
HTML_FILE_PATH = '/content/drive/MyDrive/chatgpt_export0.html'   # <- 修改
OUTPUT_MID_EXCEL = '/content/drive/MyDrive/codeblocks_with_meta.xlsx'   # 中间表输出
OUTPUT_FINAL_EXCEL = '/content/drive/MyDrive/final_original_translation_analysis.xlsx'  # 最终三列输出

# ---------------------------
# 读取 HTML（容错：二进制读 -> utf-8 decode 忽略错误）
# ---------------------------
with open(HTML_FILE_PATH, 'rb') as f:
    raw = f.read()
html_text = raw.decode('utf-8', errors='ignore')

# ---------------------------
# 用 BeautifulSoup 解析
# ---------------------------
soup = BeautifulSoup(html_text, 'lxml')

# ---------------------------
# 识别页面中“代码块”的策略（按文档出现顺序）
# 优先选择：<pre>、<code>、class含language-plaintext/whitespace-pre 的 <div> 等
# ---------------------------
def collect_code_blocks(soup):
    blocks = []
    # collect tags in document order by searching for these tag types
    # We'll iterate the DOM tree and pick nodes that match our criteria
    for tag in soup.find_all(True):
        name = tag.name.lower()
        # 1) pre or code
        if name in ('pre', 'code'):
            text = tag.get_text()
            if text and text.strip():
                blocks.append(text)
            continue
        # 2) div/span with class indicating code/plaintext
        classes = tag.get('class') or []
        classes_join = ' '.join(classes).lower() if classes else ''
        if 'language-plaintext' in classes_join or 'whitespace-pre' in classes_join or 'language-plain' in classes_join or 'language-plaintext' in classes_join:
            text = tag.get_text()
            if text and text.strip():
                blocks.append(text)
            continue
    return blocks

code_blocks = collect_code_blocks(soup)

# 如果没有识别到任何代码块，作为回退，提取所有 <div> 文本但这是非常宽松的
if not code_blocks:
    for div in soup.find_all('div'):
        t = div.get_text()
        if t and t.strip():
            code_blocks.append(t)

# 构造中间 DataFrame：代码块、页码、类别、其他（初始化后三列为空）
df_mid = pd.DataFrame({
    '代码块': code_blocks,
    '页码': [''] * len(code_blocks),
    '类别': [''] * len(code_blocks),
    '其他': [''] * len(code_blocks)
})

# ---------------------------
# 检测并拆分含“第xx页” header 的代码块
# 规则实现：
#  - 在单个代码块行内，去除空白后若某行以 "第<digits>页" 或 "第<digits>頁" 开头，则视为 header
#  - 将 header 行之后的内容作为一个新块，并为该新块设置 页码=xx，其他=header原文
#  - header 前的部分（若有）作为独立块保留（页码/其他为空）
#  - 如果原代码块被拆分，则其原记录被替换为拆分后的一系列记录（顺序保留）
# 支持半角与全角数字
# ---------------------------
page_header_re = re.compile(r'^\s*第\s*([0-9０-９]+)\s*[页頁]\s*', flags=re.UNICODE)

def to_ascii_digits(s):
    # 将全角数字转为半角
    trans = str.maketrans('０１２３４５６７８９', '0123456789')
    return s.translate(trans)

new_rows = []
for idx, row in df_mid.iterrows():
    block = str(row['代码块'])
    lines = block.splitlines()
    if not lines:
        new_rows.append({'代码块': block, '页码': row['页码'], '类别': row['类别'], '其他': row['其他']})
        continue

    i = 0
    pending_segments = []  # list of dicts {text, page(None or num), other}
    while i < len(lines):
        ln = lines[i]
        compact = re.sub(r'\s+', '', ln)  # remove whitespace
        m = page_header_re.match(compact)
        if m:
            num_raw = m.group(1)
            num = to_ascii_digits(num_raw)
            # collect content lines after header until next header or EOF
            j = i + 1
            seg_lines = []
            while j < len(lines):
                comp2 = re.sub(r'\s+', '', lines[j])
                if page_header_re.match(comp2):
                    break
                seg_lines.append(lines[j])
                j += 1
            seg_text = '\n'.join(seg_lines).strip()
            # create segment with page and other=header original text
            pending_segments.append({'代码块': seg_text, '页码': num, '类别': '', '其他': ln.strip()})
            i = j  # continue from next header or EOF
        else:
            # lines until next header go into a normal segment (no page)
            j = i
            seg_lines = []
            while j < len(lines):
                comp2 = re.sub(r'\s+', '', lines[j])
                if page_header_re.match(comp2):
                    break
                seg_lines.append(lines[j])
                j += 1
            seg_text = '\n'.join(seg_lines).strip()
            if seg_text:
                pending_segments.append({'代码块': seg_text, '页码': '', '类别': '', '其他': ''})
            i = j
    # If we didn't find any header, pending_segments will contain the same block as single entry
    # If the original block had no header (pending_segments len == 1 and page empty and other empty), treat as original
    # Append pending_segments to new_rows
    if pending_segments:
        # if the original had no header and pending_segments is single identical part, preserve original (no deletion necessary)
        # but spec says if original is split (has header) delete original and replace — that's what we do: simply add pending_segments
        # If original had no header at all, pending_segments length likely 1 with same content, we still add it
        for seg in pending_segments:
            # if segment text is empty, skip it
            if seg['代码块'] is None:
                continue
            seg_text = seg['代码块'].strip()
            if seg_text == '':
                # If blank, still might need to create a placeholder with page/other (if page exists)
                if seg['页码'] or seg['其他']:
                    new_rows.append({'代码块': '', '页码': seg['页码'], '类别': '', '其他': seg['其他']})
                continue
            new_rows.append({'代码块': seg_text, '页码': seg['页码'], '类别': '', '其他': seg['其他']})
    else:
        # fallback: keep original whole block
        new_rows.append({'代码块': block.strip(), '页码': row['页码'], '类别': '', '其他': row['其他']})

# 构建新的中间表（已拆分）
df_mid = pd.DataFrame(new_rows, columns=['代码块', '页码', '类别', '其他'])

# ---------------------------
# 根据代码块内容判断 类别（覆盖规则见说明）
# 判定规则（按你最新描述）：
# 1) 如果 代码块 同时包含 '【原文】' 字样 且 包含日语假名（平假名或片假名） => '逐行解析'
# 2) 否则若 代码块 包含日语假名 OR 全为英文字母（忽略标点空格） => '原文'
# 3) 否则 => '中文翻译'
# 但若 '其他' 列中包含关键词，则以 '其他' 的关键词覆盖类别：
#    若 '其他' 含 '原文' => '原文'; 含 '翻译' => '中文翻译'; 含 '逐行' => '逐行解析'
# ---------------------------
cn_re = re.compile(r'[\u4e00-\u9fff]')
jp_re = re.compile(r'[\u3040-\u30ff\u31f0-\u31ff]')  # hiragana, katakana ranges
# ASCII letter check: consider "全是英文字母" -> after removing spaces and punctuation, remaining are A-Za-z
import re

# 日语平假名和片假名范围
jp_re = re.compile(r'[\u3040-\u309F\u30A0-\u30FF\u31F0-\u31FF]')

# 判断是否全为 ASCII 英文字母、空格和常见标点
ascii_letters_re = re.compile(r'^[A-Za-z\s!"#$%&\'()*+,\-./:;<=>?@[\\\]^_`{|}~]+$')

def is_all_ascii_letters(text):
    # 去除换行等控制字符
    text = text.strip()
    if not text:
        return False
    return bool(ascii_letters_re.fullmatch(text))

# 测试：
print(is_all_ascii_letters("This is a test, with punctuation!"))  # True
print(is_all_ascii_letters("これはテストです"))  # False


def detect_kind(block_text, other_text):
    txt = block_text or ''
    other = other_text or ''
    other_low = other.lower()
    # 优先检查“其他”列覆盖关键词
    if any(k in other_low for k in ['逐行', '逐行解析', '逐行詳細解析']):
        return '逐行解析'
    if '翻' in other_low or '翻译' in other_low or '翻譯' in other_low:
        return '中文翻译'
    if '原文' in other_low:
        return '原文'

    # 主要判定逻辑修改如下：
    #has_marker_gen = ('【原文】' in txt)
    has_marker_fa = ('【发音】' in txt)
    has_jp = bool(jp_re.search(txt))

    #if (has_marker_gen and has_marker_fa) or ('逐行' in txt):
    if has_marker_fa or ('逐行' in txt):
        return '逐行解析'
    if has_jp or is_all_ascii_letters(txt):
        return '原文'

    return '中文翻译'

# Apply detection
kinds = []
for idx, row in df_mid.iterrows():
    k = detect_kind(str(row['代码块']), str(row['其他']))
    kinds.append(k)
df_mid['类别'] = kinds

# ---------------------------
# 合并连续的 “逐行解析” 代码块（合并所有连续块）
# ---------------------------
merged_rows = []
i = 0
n = len(df_mid)
while i < n:
    row = df_mid.iloc[i]
    if row['类别'] == '逐行解析':
        segs = [str(row['代码块']).strip()]
        pages = [str(row['页码']).strip()] if str(row['页码']).strip() else []
        others = [str(row['其他']).strip()] if str(row['其他']).strip() else []
        j = i + 1
        while j < n and df_mid.iloc[j]['类别'] == '逐行解析':
            segs.append(str(df_mid.iloc[j]['代码块']).strip())
            if str(df_mid.iloc[j]['页码']).strip():
                pages.append(str(df_mid.iloc[j]['页码']).strip())
            if str(df_mid.iloc[j]['其他']).strip():
                others.append(str(df_mid.iloc[j]['其他']).strip())
            j += 1
        merged_text = '\n'.join([s for s in segs if s])
        merged_page = pages[0] if pages else ''
        merged_other = '\n'.join([o for o in others if o])
        merged_rows.append({'代码块': merged_text, '页码': merged_page, '类别': '逐行解析', '其他': merged_other})
        i = j
    else:
        merged_rows.append({'代码块': row['代码块'], '页码': row['页码'], '类别': row['类别'], '其他': row['其他']})
        i += 1

df_mid = pd.DataFrame(merged_rows, columns=['代码块', '页码', '类别', '其他'])

# ---------------------------
# 基于中间表生成最终三列表：原文 / 中文翻译 / 逐行解析
# 规则：遍历中间表，遇到 类别 == '原文' 时创建一行，尝试把其后紧跟的 翻译/解析 填入对应列
# 注意：如果翻译或解析不存在，则为空。之后去重合并相同原文（见下）
# ---------------------------
final_rows = []
i = 0
n = len(df_mid)
while i < n:
    row = df_mid.iloc[i]
    if row['类别'] == '原文':
        orig = str(row['代码块']).strip()
        trans = ''
        analysis = ''
        # look ahead
        if i + 1 < n and df_mid.iloc[i+1]['类别'] == '中文翻译':
            trans = str(df_mid.iloc[i+1]['代码块']).strip()
            # maybe next is 逐行解析
            if i + 2 < n and df_mid.iloc[i+2]['类别'] == '逐行解析':
                analysis = str(df_mid.iloc[i+2]['代码块']).strip()
                i += 2
            else:
                i += 1
        elif i + 1 < n and df_mid.iloc[i+1]['类别'] == '逐行解析':
            analysis = str(df_mid.iloc[i+1]['代码块']).strip()
            i += 1
        final_rows.append({'原文': orig, '中文翻译': trans, '逐行解析': analysis})
    i += 1

df_final = pd.DataFrame(final_rows, columns=['原文', '中文翻译', '逐行解析'])

# ---------------------------
# 对 df_final 按 '原文' 去重，保留一条，并取最长的 翻译/解析 字段
# ---------------------------
def choose_longest(series):
    vals = [str(x) for x in series if str(x).strip()]
    return max(vals, key=len) if vals else ''

if not df_final.empty:
    df_grouped = df_final.groupby('原文', as_index=False).agg({
        '中文翻译': choose_longest,
        '逐行解析': choose_longest
    })
else:
    df_grouped = df_final.copy()

# ---------------------------
# 导出中间表和最终表
# ---------------------------
df_mid.to_excel(OUTPUT_MID_EXCEL, index=False)
df_grouped.to_excel(OUTPUT_FINAL_EXCEL, index=False)

print("完成。")
print("中间表（代码块、页码、类别、其他）保存为:", OUTPUT_MID_EXCEL)
print("最终表（原文、中文翻译、逐行解析）保存为:", OUTPUT_FINAL_EXCEL)

True
False
完成。
中间表（代码块、页码、类别、其他）保存为: /content/drive/MyDrive/codeblocks_with_meta.xlsx
最终表（原文、中文翻译、逐行解析）保存为: /content/drive/MyDrive/final_original_translation_analysis.xlsx


In [ ]:
!pip install beautifulsoup4 lxml pandas openpyxl

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 运行前请确保安装依赖（Colab可执行）
# !pip install beautifulsoup4 lxml pandas openpyxl

import re
import pandas as pd
from bs4 import BeautifulSoup

# ---------------------------
# 配置：将下面路径替换为你的文件路径
# ---------------------------
#HTML_FILE_PATH = '/content/drive/MyDrive/chatgpt_export0.html'      # <- 修改
#HTML_FILE_PATH = '/content/drive/MyDrive/chatgpt_export.html'      # <- 修改
HTML_FILE_PATH = '/content/drive/MyDrive/chatgpt_export180_350.html'      # <- 修改
#HTML_FILE_PATH = '/content/drive/MyDrive/chatgpt_export431_770.html'      # <- 修改
OUTPUT_MID_MERGED_EXCEL = '/content/drive/MyDrive/codeblocks_with_meta_merged.xlsx'  # 中间表输出
OUTPUT_MID_EXCEL = '/content/drive/MyDrive/codeblocks_with_meta.xlsx'  # 中间表输出
OUTPUT_FINAL_EXCEL = '/content/drive/MyDrive/final_original_translation_analysis.xlsx'  # 最终三列输出
OUTPUT_FINAL_FORMAT_EXCEL = '/content/drive/MyDrive/final_original_translation_analysis_format.xlsx'  # 最终三列输出
OUTPUT_FINAL_FORMAT_CSV = '/content/drive/MyDrive/final_original_translation_analysis_format.csv' # 最终三列输出

# ---------------------------
# 读取 HTML（容错：二进制读 -> utf-8 decode 忽略错误）
# ---------------------------
with open(HTML_FILE_PATH, 'rb') as f:
    raw = f.read()
html_text = raw.decode('utf-8', errors='ignore')

# ---------------------------
# 用 BeautifulSoup 解析
# ---------------------------
soup = BeautifulSoup(html_text, 'lxml')

# ---------------------------
# 识别页面中“代码块”的策略（按文档出现顺序）
# 优先选择：<pre>、<code>、class含language-plaintext/whitespace-pre 的 <div> 等
# ---------------------------
def clean_pre_text(text):
    """
    清理 <pre> 标签内的多余文字，例如开头的“plaintext”、“复制代码”等。
    """
    remove_phrases = ['plaintext', '复制代码', 'コピーコード']
    for phrase in remove_phrases:
        text = text.replace(phrase, '')
    # 去除开头结尾多余空白符
    return text.lstrip('\n\r ').rstrip()

def collect_code_blocks(soup):
    blocks = []
    # collect tags in document order by searching for these tag types
    for tag in soup.find_all(True):
        name = tag.name.lower()
        if name in ('pre', 'code'):
            text = tag.get_text()
            if text and text.strip():
                blocks.append(text)
            continue
        classes = tag.get('class') or []
        classes_join = ' '.join(classes).lower() if classes else ''
        if ('language-plaintext' in classes_join or 'whitespace-pre' in classes_join
            or 'language-plain' in classes_join):
            text = tag.get_text()
            if text and text.strip():
                blocks.append(text)
            continue
    return blocks

code_blocks = collect_code_blocks(soup)

# 如果没有识别到任何代码块，作为回退，提取所有 <div> 文本
if not code_blocks:
    for div in soup.find_all('div'):
        t = div.get_text()
        if t and t.strip():
            code_blocks.append(t)

# 构造中间 DataFrame：代码块、页码、类别、其他（初始化后三列为空）
df_mid = pd.DataFrame({
    '代码块': code_blocks,
    '页码': [''] * len(code_blocks),
    '类别': [''] * len(code_blocks),
    '其他': [''] * len(code_blocks)
})

# ---------------------------
# 检测并拆分含“第xx页” header 的代码块
# 规则：
# - 行内去空白，若行以 "第<digits>页" 或 "第<digits>頁" 开头视为 header
# - header 之后内容作为新块，页码=xx，其他=header原文
# - header 前的部分（若有）单独作为块，页码/其他为空
# - 原代码块被拆分，则替换成拆分后多条记录
# ---------------------------
page_header_re = re.compile(r'^\s*第\s*([0-9０-９]+)\s*[页頁]\s*', flags=re.UNICODE)

def to_ascii_digits(s):
    trans = str.maketrans('０１２３４５６７８９', '0123456789')
    return s.translate(trans)

new_rows = []
global_seq = 1  # 总顺序

for idx, row in df_mid.iterrows():
    block = clean_pre_text(str(row['代码块']))
    lines = block.splitlines()
    if not lines:
        new_rows.append({'代码块': block, '页码': row['页码'], '类别': row['类别'], '其他': row['其他'],
                         '顺序': global_seq, '子顺序': 1})
        global_seq += 1
        continue

    i = 0
    pending_segments = []
    sub_seq = 1  # 子顺序

    while i < len(lines):
        ln = lines[i]
        compact = re.sub(r'\s+', '', ln)
        m = page_header_re.match(compact)
        if m:
            num_raw = m.group(1)
            num = to_ascii_digits(num_raw)
            j = i + 1
            seg_lines = []
            while j < len(lines):
                comp2 = re.sub(r'\s+', '', lines[j])
                if page_header_re.match(comp2):
                    break
                seg_lines.append(lines[j])
                j += 1
            seg_text = '\n'.join(seg_lines).strip()
            if seg_text:
                pending_segments.append({'代码块': clean_pre_text(seg_text), '页码': num, '类别': '', '其他': ln.strip(),
                                         '顺序': global_seq, '子顺序': sub_seq})
                sub_seq += 1
            i = j
        else:
            j = i
            seg_lines = []
            while j < len(lines):
                comp2 = re.sub(r'\s+', '', lines[j])
                if page_header_re.match(comp2):
                    break
                seg_lines.append(lines[j])
                j += 1
            seg_text = '\n'.join(seg_lines).strip()
            if seg_text:
                pending_segments.append({'代码块': clean_pre_text(seg_text), '页码': '', '类别': '', '其他': '',
                                         '顺序': global_seq, '子顺序': sub_seq})
                sub_seq += 1
            i = j

    if pending_segments:
        for seg in pending_segments:
            new_rows.append(seg)
        global_seq += 1
    else:
        new_rows.append({'代码块': clean_pre_text(block.strip()), '页码': row['页码'], '类别': row['类别'], '其他': row['其他'],
                         '顺序': global_seq, '子顺序': 1})
        global_seq += 1

df_mid = pd.DataFrame(new_rows, columns=['代码块', '页码', '类别', '其他', '顺序', '子顺序'])

# ---------------------------
# 类别判定
# ---------------------------
jp_re = re.compile(r'[\u3040-\u309F\u30A0-\u30FF\u31F0-\u31FF]')  # 日语平假名片假名
ascii_letters_re = re.compile(r'^[A-Za-z\s!"#$%&\'()*+,\-./:;<=>?@[\\\]^_`{|}~]+$')

def is_all_ascii_letters(text):
    text = text.strip()
    if not text:
        return False
    return bool(ascii_letters_re.fullmatch(text))

def detect_kind(block_text, other_text):
    txt = block_text or ''
    other = other_text or ''
    other_low = other.lower()
    if any(k in other_low for k in ['逐行', '逐行解析', '逐行詳細解析']):
        return '逐行解析'
    if '翻' in other_low or '翻译' in other_low or '翻譯' in other_low:
        return '中文翻译'
    if '原文' in other_low:
        return '原文'
    has_marker_fa = ('发音' in txt) or ('発音' in txt)
    has_marker_Chinese = ('中文翻译' in txt)

    has_jp = bool(jp_re.search(txt))
    if has_marker_Chinese:
        return '中文翻译'

    if has_marker_fa or ('逐行' in txt):
        return '逐行解析'

    if has_jp or is_all_ascii_letters(txt) or (' 原文' in txt):
        return '原文'


    return '中文翻译'

kinds = []
for idx, row in df_mid.iterrows():
    k = detect_kind(str(row['代码块']), str(row['其他']))
    kinds.append(k)
df_mid['类别'] = kinds

# ---------------------------
# 合并连续“逐行解析”代码块，顺序取最小，子顺序取最小
# ---------------------------
merged_rows = []
i = 0
n = len(df_mid)
while i < n:
    row = df_mid.iloc[i]
    if row['类别'] == '逐行解析':
        segs = [str(row['代码块']).strip()]
        pages = [str(row['页码']).strip()] if str(row['页码']).strip() else []
        others = [str(row['其他']).strip()] if str(row['其他']).strip() else []
        seqs = [row['顺序']]
        subs = [row['子顺序']]
        j = i + 1
        while j < n and df_mid.iloc[j]['类别'] == '逐行解析':
            segs.append(str(df_mid.iloc[j]['代码块']).strip())
            if str(df_mid.iloc[j]['页码']).strip():
                pages.append(str(df_mid.iloc[j]['页码']).strip())
            if str(df_mid.iloc[j]['其他']).strip():
                others.append(str(df_mid.iloc[j]['其他']).strip())
            seqs.append(df_mid.iloc[j]['顺序'])
            subs.append(df_mid.iloc[j]['子顺序'])
            j += 1
        merged_text = '\n'.join([s for s in segs if s])
        merged_page = pages[0] if pages else ''
        merged_other = '\n'.join([o for o in others if o])
        merged_seq = min(seqs)
        merged_sub = min(subs)
        merged_rows.append({'代码块': merged_text, '页码': merged_page, '类别': '逐行解析',
                            '其他': merged_other, '顺序': merged_seq, '子顺序': merged_sub})
        i = j
    else:
        merged_rows.append({'代码块': row['代码块'], '页码': row['页码'], '类别': row['类别'],
                            '其他': row['其他'], '顺序': row['顺序'], '子顺序': row['子顺序']})
        i += 1
df_mid = pd.DataFrame(merged_rows, columns=['代码块', '页码', '类别', '其他', '顺序', '子顺序'])

# ...完成所有拆分、类别判定和合并代码块后

# 删除除顺序和子顺序外内容重复的记录，只保留最后出现的
cols_to_check = ['代码块', '页码', '类别', '其他']
df_mid = df_mid.drop_duplicates(subset=cols_to_check, keep='last').reset_index(drop=True)

df_mid.to_excel(OUTPUT_MID_MERGED_EXCEL, index=False)

# 然后继续后续处理，比如生成最终表...

# ---------------------------
# 基于中间表生成最终三列表：原文 / 中文翻译 / 逐行解析
# 规则：
# 遍历中间表，遇到 类别 == '原文' 时创建一行，
# 尝试把其后紧跟的 翻译/解析 填入对应列
# 注意：如果翻译或解析不存在，则为空。
# 之后对相同原文去重，保留顺序最早（顺序最小）的记录，
# 翻译、逐行解析保留最长的内容
# ---------------------------

final_rows = []
i = 0
n = len(df_mid)

while i < n:
    row = df_mid.iloc[i]
    if row['类别'] == '原文':
        orig = str(row['代码块']).strip()
        trans = ''
        analysis = ''
        orig_seq = row['顺序']
        orig_sub = row['子顺序']

        # 向后寻找“中文翻译”或“逐行解析”
        if i + 1 < n and df_mid.iloc[i + 1]['类别'] == '中文翻译':
            trans = str(df_mid.iloc[i + 1]['代码块']).strip()
            if i + 2 < n and df_mid.iloc[i + 2]['类别'] == '逐行解析':
                analysis = str(df_mid.iloc[i + 2]['代码块']).strip()
                i += 2
            else:
                i += 1
        elif i + 1 < n and df_mid.iloc[i + 1]['类别'] == '逐行解析':
            analysis = str(df_mid.iloc[i + 1]['代码块']).strip()
            i += 1

        final_rows.append({
            '原文': orig,
            '中文翻译': trans,
            '逐行解析': analysis,
            '顺序': orig_seq,
            '子顺序': orig_sub
        })
    i += 1

df_final = pd.DataFrame(final_rows, columns=['原文', '中文翻译', '逐行解析', '顺序', '子顺序'])

# ---------------------------
# 按“原文”去重，保留顺序最早的记录，
# 翻译和逐行解析取最长文本
# ---------------------------
def choose_longest(series):
    vals = [str(x) for x in series if str(x).strip()]
    return max(vals, key=len) if vals else ''

if not df_final.empty:
    # 先排序，保证顺序最小优先
    df_final = df_final.sort_values(by=['顺序', '子顺序'])
    # 按“原文”聚合，保留第一条记录的顺序和子顺序，合并最长文本
    df_grouped = df_final.groupby('原文', as_index=False).agg({
        '顺序': 'first',
        '子顺序': 'first',
        '中文翻译': choose_longest,
        '逐行解析': choose_longest
    })
    # 结果再按顺序子顺序排序
    df_grouped = df_grouped.sort_values(by=['顺序', '子顺序']).reset_index(drop=True)
else:
    df_grouped = df_final.copy()

# ---------------------------
# 导出中间表和最终表
# ---------------------------
df_mid.to_excel(OUTPUT_MID_EXCEL, index=False)
df_grouped[['原文', '中文翻译', '逐行解析']].to_excel(OUTPUT_FINAL_EXCEL, index=False)


# 删除“中文翻译”和“逐行解析”两列同时为空的记录
import re
import pandas as pd

# 假设 df_final 已经生成，含列 ['原文','中文翻译','逐行解析', ...]
# 如果你的 DataFrame 叫 df_grouped 或其它，把名字替换过来即可

def normalize_cell_to_NA(x):
    """
    把各种“空”形式规范为 pd.NA：
    - None/NaN/''/只含空白/全角空格/'nan'/'None'/'null'/'-' 等 -> pd.NA
    - 其它字符串会去除首尾空白并返回
    """
    if pd.isna(x):
        return pd.NA
    s = str(x)
    # 去除常规空白与全角空格（两端）
    s = re.sub(r'^[\s\u3000]+|[\s\u3000]+$', '', s)
    # 如果变成空字符串或常见占位字符串，视为缺失
    if s == '' or s.lower() in ('nan', 'none', 'null', '-'):
        return pd.NA
    return s

def drop_rows_both_translation_empty(df, col1='中文翻译', col2='逐行解析', inplace=False):
    """
    规范化 col1,col2 并删除同时为空的行
    返回 (df_new, removed_count)
    """
    if not inplace:
        df = df.copy()

    # 统计删除前数量
    before = len(df)

    # 规范化两列（如果没有列会抛出，确保列名正确）
    df[[col1, col2]] = df[[col1, col2]].applymap(normalize_cell_to_NA)

    # debug: 看看到底哪些行被识别为空（打印样例）
    mask_both_empty = df[col1].isna() | df[col2].isna()
    sample = df[mask_both_empty].head(10)
    if not sample.empty:
        print(f"示例：这 {len(sample)} 行会被删除（显示前 10 行）：")
        print(sample[['原文', col1, col2]].to_string(index=False))
    else:
        print("没有检测到同时为空的行（前 10 个样例为空）。")

    # 执行删除（两列同时为空）
    df = df[~mask_both_empty].reset_index(drop=True)
    after = len(df)
    removed = before - after
    print(f"已删除 {removed} 行；剩余 {after} 行（原始 {before} 行）。")
    return df, removed

# 用法示例（假设 df_final 是目标 DataFrame）：
df_final, removed_count = drop_rows_both_translation_empty(df_grouped[['原文', '中文翻译', '逐行解析']])
# 如果你想就地修改：
#df_final, _ = drop_rows_both_translation_empty(df_final)

# 如果还需要把结果写回 Excel：
# df_final.to_excel('final_filtered.xlsx', index=False)
#df_final = df_grouped[['原文', '中文翻译', '逐行解析']].dropna(subset=['中文翻译', '逐行解析'], how='all')
df_final.to_excel(OUTPUT_FINAL_FORMAT_EXCEL, index=False)


import re
import pandas as pd

def add_page_and_section(df, col_original='原文'):
    """
    在 DataFrame 第一列前添加两列：'页码'（提取自'原文'列）和'section'（空字符串）
    """
    # 提取页码
    def extract_page(s):
        if pd.isna(s):
            return ''
        m = re.search(r'第\s*(\d+)\s*页', str(s))
        if m:
            return m.group(1)
        return ''

    df = df.copy()
    page_vals = df[col_original].apply(extract_page)
    section_vals = [''] * len(df)

    # 在第一列前插入两列
    df.insert(0, 'section', section_vals)
    df.insert(0, '页码', page_vals)

    return df

# 用法示例：
# 假设 df_final 是你的 DataFrame
df_final = add_page_and_section(df_final)
#df_final.to_csv(OUTPUT_FINAL_FORMAT_CSV, index=False, encoding='utf-8-sig')

#def keep_longest_by_page(df, col_page='页码', col_target='逐行解析'):
def keep_longest_by_page(df, col_page='原文', col_target='逐行解析'):
    """
    按页码分组，只保留逐行解析列里内容最长的记录
    如果逐行解析为空，保留组里的第一条
    """
    def pick_longest(group):
        # 找到非空的最长逐行解析
        non_empty = group[group[col_target].astype(str).str.strip() != '']
        if len(non_empty) > 0:
            # 选最长的那条
            idx_max = non_empty[col_target].astype(str).str.len().idxmax()
            return group.loc[[idx_max]]
        else:
            # 全部为空，保留第一条
            return group.iloc[[0]]

    return pd.concat([pick_longest(g) for _, g in df.groupby(col_page)], ignore_index=True)

# 用法示例：
df_final = keep_longest_by_page(df_final, col_page='页码', col_target='逐行解析')
df_final.to_csv(OUTPUT_FINAL_FORMAT_CSV, index=False, encoding='utf-8-sig')
df_final.to_excel(OUTPUT_FINAL_EXCEL, index=False)

print("完成。")
print("中间表（代码块、页码、类别、其他、顺序、子顺序）保存为:", OUTPUT_MID_EXCEL)
print("最终表（原文、中文翻译、逐行解析）保存为:", OUTPUT_FINAL_EXCEL)

/tmp/ipython-input-1833413538.py:365: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[[col1, col2]] = df[[col1, col2]].applymap(normalize_cell_to_NA)


示例：这 10 行会被删除（显示前 10 行）：
                                                                                                                                                                                                                                                                                                                                 原文 中文翻译 逐行解析
                                                                                                                                         OK\n🟩 第27页内容：多要素認証 (Multi Factor Authentication - MFA)• MFA = パスワード(you know) + セキュリティデバイス(you have)• MFA の主なメリット: もしパスワードが盗まれたりハックされても、アカウントが保 護されます Alice + Password => Successful login <NA> <NA>
                                                                                                                 🟩 第28页内容：\nAWS で使えるMFA デバイス バーチャルMFA デバイス Google Authenticator (スマホのみ) Authy (様々なデバイス) Universal 2nd Factor (U2F) Security Key YubiKey by Yubico (3rd party) 一つのデバイスで複数トークンをサポート 一つのセキュリティキーで、複数の 

In [ ]:

import pandas as pd
import re
from bs4 import BeautifulSoup
from difflib import SequenceMatcher

# ===== 文本清理函数 =====
def remove_html_tags(text):
    if pd.isna(text):
        return ""
    soup = BeautifulSoup(str(text), "html.parser")
    return soup.get_text(separator=" ")

def clean_text(text):
    text = remove_html_tags(text)
    text = re.sub(r"^\s*(🟩\s*)?第\d+页内容[:：]", "", text)
    text = text.replace("【原文】", "")
    text = re.sub(r"\s+", " ", text).strip()
    return text

# ===== 读取数据 =====
csv_file = "/content/drive/MyDrive/aws_pages.csv"
xlsx_file = "/content/drive/MyDrive/anki_cleaned.xlsx"

df_csv = pd.read_csv(csv_file, usecols=["页码", "原文"])
df_xlsx = pd.read_excel(xlsx_file, usecols=["原文", "中文翻译", "逐行解析"])

# 转换页码为数字
df_csv["页码"] = pd.to_numeric(df_csv["页码"], errors="coerce")

# 清理
df_csv["原文_clean"] = df_csv["原文"].apply(clean_text)
df_xlsx["原文_clean"] = df_xlsx["原文"].apply(clean_text)

# 初始化新 DataFrame
df_new = pd.DataFrame({
    "页码": [None] * len(df_xlsx),
    "section": ["" for _ in range(len(df_xlsx))],
    "原文": df_xlsx["原文"],
    "中文翻译": df_xlsx["中文翻译"],
    "逐行解析": df_xlsx["逐行解析"]
})

# 匹配
for idx, row in df_xlsx.iterrows():
    target_text = row["原文_clean"]
    if not target_text:
        continue

    best_score = 0
    best_page = None

    for _, csv_row in df_csv.iterrows():
        score = SequenceMatcher(None, target_text, csv_row["原文_clean"]).ratio()
        if score > best_score:
            best_score = score
            best_page = csv_row["页码"]

    # 只在相似度达到阈值时才填充
    if best_score > 0.75:
        df_new.at[idx, "页码"] = best_page

# 保存
output_file = "/content/drive/MyDrive/similar_check.xlsx"
df_new.to_excel(output_file, index=False)
print(f"✅ similar_check.xlsx 已生成: {output_file}")

✅ similar_check.xlsx 已生成: /content/drive/MyDrive/similar_check.xlsx


In [ ]:
#anki_csv_file = "/content/drive/MyDrive/similar_check_for_anki.csv"
anki_csv_file = "/content/drive/MyDrive/AWS_Anki.txt"

# ========== 导出成 Anki 可用 CSV ==========
#anki_csv_file = "/content/drive/MyDrive/anki_import.txt"

# 按 Anki 需求：用制表符分隔，不要表头，不要索引
df_new.to_csv(
    anki_csv_file,
    sep="\t",
    index=False,
    header=False,
    encoding="utf-8"
)

print(f"✅ 已导出可直接导入 Anki 的 txt 文件：{anki_csv_file}")
#df_new.to_csv(anki_csv_file, index=False, encoding="utf-8-sig")
#print(f"已生成 Anki 导入 CSV：{anki_csv_file}")

✅ 已导出可直接导入 Anki 的 txt 文件：/content/drive/MyDrive/AWS_Anki.txt


In [ ]:

import pandas as pd
import re
from bs4 import BeautifulSoup
from difflib import SequenceMatcher

#anki_txt_file = "/content/drive/MyDrive/AWS_Anki.txt"
anki_txt_file = "/content/drive/MyDrive/AWS_Anki_total.txt"
output_excel = "/content/drive/MyDrive/anki_cleaned.xlsx"
output_csv = "/content/drive/MyDrive/anki_cleaned.csv"
output_original_csv = "/content/drive/MyDrive/anki_original_cleaned.csv"
output_translation_csv = "/content/drive/MyDrive/anki_translation_cleaned.csv"
output_original_translation_csv = "/content/drive/MyDrive/anki_original_translation_cleaned.csv"

def remove_html_tags(text):
    if pd.isna(text):
        return ""
    soup = BeautifulSoup(str(text), "html.parser")
    return soup.get_text(separator=" ")  # 用空格替换换行等

def remove_invisible_chars(text):
    invisible_chars = [
        '\u200b',  # 零宽空格
        '\ufeff',  # BOM
        '\u3000',  # 全角空格
    ]
    for ch in invisible_chars:
        text = text.replace(ch, '')
    return text

import unicodedata

def normalize_text(text):
    # NFKC标准化，统一全角半角、特殊字符等
    text = unicodedata.normalize('NFKC', text)
    # 去除多余空格（词间只留一个空格）
    text = re.sub(r'\s+', ' ', text)
    # 去除空格前后的特殊字符空格
    text = re.sub(r'\s+([、。！？])', r'\1', text)
    return text

def clean_original(text):
    text = remove_html_tags(text)
    text = re.sub(r"(🟩\s*)?第\d+页(?:内容[:：]| 原文\s*)", "", text)
    text = text.replace("【原文】", "")
    text = remove_invisible_chars(text)
    text = normalize_text(text)
    text = text.strip()
    return text

def clean_text_field(text):
    if pd.isna(text):
        return ""
    text = remove_html_tags(text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

def merge_similar_rows(group, threshold=0.85):
    rows = group.copy()
    texts = rows["原文_clean"].tolist()
    n = len(texts)
    labels = [-1] * n
    current_label = 0
    for i in range(n):
        if labels[i] == -1:
            labels[i] = current_label
            for j in range(i + 1, n):
                if labels[j] == -1:
                    if similar(texts[i], texts[j]) >= threshold:
                        labels[j] = current_label
            current_label += 1
    rows["group_label"] = labels

    def select_longest_subgroup(subgroup):
        max_len = subgroup["逐行解析_clean"].str.len().max()
        longest_rows = subgroup[subgroup["逐行解析_clean"].str.len() == max_len]
        return longest_rows.iloc[0]

    result = rows.groupby("group_label").apply(select_longest_subgroup)
    return result.reset_index(drop=True)

# 读取anki导出txt
df = pd.read_csv(
    anki_txt_file, sep="\t", header=None, usecols=[0,1,2,3,4],
    names=["页码", "section", "原文", "中文翻译", "逐行解析"]
)
df["页码"] = pd.to_numeric(df["页码"], errors="coerce")
df = df.drop_duplicates()

df["原文_clean"] = df["原文"].apply(clean_original)
df["中文翻译_clean"] = df["中文翻译"].apply(clean_text_field)
df["逐行解析_clean"] = df["逐行解析"].apply(clean_text_field)
df["len_sum"] = df["中文翻译_clean"].str.len() + df["逐行解析_clean"].str.len()

# 按“页码+原文_clean”分组，保留len_sum最长的
grouped = df.groupby(["页码", "原文_clean"])
result_rows = []
for name, group in grouped:
    max_len = group["len_sum"].max()
    max_row = group[group["len_sum"] == max_len].iloc[0]
    result_rows.append(max_row)
df_cleaned = pd.DataFrame(result_rows)

# 删除同原文中逐行解析为空的（只保留非空版本）
df_cleaned["逐行解析_is_empty"] = df_cleaned["逐行解析_clean"].str.strip() == ""
mask_to_drop = (
    df_cleaned.groupby("原文_clean")["逐行解析_is_empty"]
    .transform(lambda x: x.any() and x.sum() < len(x))
    & df_cleaned["逐行解析_is_empty"]
)
df_cleaned = df_cleaned[~mask_to_drop].copy()
df_cleaned = df_cleaned.drop(columns=["逐行解析_is_empty"])

# 合并相似原文（只在同一页码内合并）
df_merged = df_cleaned.groupby("页码", group_keys=False).apply(merge_similar_rows)

# 检测同一页码下原文不同
conflicts = df_merged.groupby("页码")["原文_clean"].nunique()
conflict_pages = conflicts[conflicts > 1]
if not conflict_pages.empty:
    print("⚠️ 合并相似原文后，仍检测到同一页码下有多个不同原文：")
    for page, count in conflict_pages.items():
        print(f"  页码 {page} 有 {count} 条不同原文：")
        originals = df_merged[df_merged["页码"] == page]["原文_clean"].unique()
        for idx, orig in enumerate(originals, 1):
            print(f"    {idx}. {orig[:60]}{'...' if len(orig) > 60 else ''}")

df_merged = df_merged.sort_values(by="页码", ascending=True, na_position="last")

# 假设最终的 DataFrame 是 df_merged

cols_to_export = ["页码", "section", "原文", "中文翻译", "逐行解析"]

df_merged.to_excel(output_excel, columns=cols_to_export, index=False)
df_merged.to_csv(output_csv, columns=cols_to_export, index=False, encoding="utf-8-sig")

print(f"✅ 处理完成并已按页码排序，已导出到：\n{output_excel}\n{output_csv}")

def check_duplicates_and_empty_parsing(df):
    # 检测重复页码
    page_counts = df["页码"].value_counts()
    duplicate_pages = page_counts[page_counts > 1]

    if not duplicate_pages.empty:
        print("⚠️ 存在重复的页码：")
        for page, count in duplicate_pages.items():
            print(f"  页码 {page} 出现了 {count} 次")
    else:
        print("✅ 没有重复的页码。")

    # 检测逐行解析为空
    empty_parsing_rows = df[df["逐行解析"].str.strip() == ""]
    if not empty_parsing_rows.empty:
        print("⚠️ 以下记录的‘逐行解析’字段为空：")
        print(empty_parsing_rows[["页码", "原文"]])
    else:
        print("✅ 所有记录‘逐行解析’字段均非空。")

# 调用示例
check_duplicates_and_empty_parsing(df_merged)

# 假设最大页码为数据中的最大页码，也可以自己指定
max_page = int(df_merged["页码"].max())

expected_pages = set(range(1, max_page + 1))
existing_pages = set(df_merged["页码"].dropna().astype(int).unique())

missing_pages = expected_pages - existing_pages

if missing_pages:
    print("⚠️ 缺失的页码有：")
    print(sorted(missing_pages))
else:
    print("✅ 没有缺失页码。")


df_merged.to_csv(output_original_csv, columns=["原文"], index=False, encoding="utf-8-sig")
df_merged.to_csv(output_translation_csv, columns=["中文翻译"], index=False, encoding="utf-8-sig")
df_merged.to_csv(output_original_translation_csv, columns=["原文", "中文翻译"], index=False, encoding="utf-8-sig")

In [ ]:
# ---------------------------
# 安装依赖（Colab 环境）
# ---------------------------
!pip install -q chardet beautifulsoup4 markdown pygments

# ---------------------------
# 导入与配置
# ---------------------------
import os
import chardet
from bs4 import BeautifulSoup
import markdown
from markdown.extensions import fenced_code, tables, attr_list
import html
import sys
from pathlib import Path
from datetime import datetime

# 修改这里：
# 输入文件夹（你的导出 html 文件放在这里），相对路径示例： "drive/MyDrive/chatgpt_exports"
# 输出文件夹（生成的渲染 html 会放在这里）
INPUT_FOLDER = "/content/drive/MyDrive/chatgpt_exports"   # <-- 修改为你 Drive 中的输入文件夹
OUTPUT_FOLDER = "/content/drive/MyDrive/chatgpt_html_rendered_v4"  # <-- 修改为输出文件夹

# ---------------------------
# 挂载 Google Drive（如果还没挂载）
# ---------------------------
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# ---------------------------
# 工具函数：安全读取并检测编码
# ---------------------------
def read_file_with_encoding_detection(path):
    with open(path, "rb") as f:
        raw = f.read()
    # chardet 检测
    res = chardet.detect(raw)
    enc = res.get("encoding") or "utf-8"
    try:
        text = raw.decode(enc, errors="ignore")
    except Exception:
        text = raw.decode("utf-8", errors="ignore")
    return text, enc

# ---------------------------
# 主要解析逻辑（尽量保留原始 HTML / 内部样式）
# ---------------------------
def extract_messages_from_html(html_text):
    """
    返回一个按顺序的消息列表，每条为 dict: {role: "user"|"chatgpt"|"system", html: "<原始内部HTML>", text: "纯文本"}
    尽量使用多重启发式规则判定角色，优先保留原始内部 HTML（decode_contents）。
    """
    soup = BeautifulSoup(html_text, "html.parser")  # 宽松解析

    messages = []

    # 常见导出里可能的容器选择器（尝试多种）
    possible_selectors = [
        {'attr':'class', 'values': ['assistant','assistant-message','message-row-assistant','chatgpt']},
        {'attr':'class', 'values': ['user','user-message','message-row-user','from-user']},
        {'attr':'id', 'values': ['assistant','user']}
    ]
    # 首先尝试找到明显带 class/id 的消息容器
    found_nodes = []
    for tag in soup.find_all(True):
        classes = [c.lower() for c in (tag.get("class") or [])]
        tag_id = (tag.get("id") or "").lower()
        is_candidate = False
        for s in possible_selectors:
            if s['attr'] == 'class':
                for v in s['values']:
                    if v in classes:
                        found_nodes.append(tag)
                        is_candidate = True
                        break
            else:
                if tag_id in s['values']:
                    found_nodes.append(tag)
                    is_candidate = True
                    break
        # 避 deep search too much here; we collect, filter below
    # 如果没找到任何带标识节点，退回到遍历 body 的直接子节点
    if not found_nodes:
        # 遍历 body 的直接子标签（保持顺序）
        body = soup.body or soup
        for tag in body.find_all(recursive=False):
            found_nodes.append(tag)

    # 再对 found_nodes 做更严格的筛查与分段
    for node in found_nodes:
        # 提取纯文本（用于判别）和内部 HTML（用于保留样式）
        text = node.get_text(separator="\n", strip=True)
        inner_html = node.decode_contents()  # 保留 node 内部的原始 HTML

        # 过滤无实质内容
        if not text or len(text.strip()) == 0:
            continue

        # 判定角色：多条件
        role = "user"  # 默认用户
        classes = [c.lower() for c in (node.get("class") or [])]
        node_id = (node.get("id") or "").lower()

        # 优先依据 class / id 关键词
        if any("assistant" in c or "chatgpt" in c for c in classes) or "assistant" in node_id:
            role = "chatgpt"
        elif any("user" in c or "from-user" in c for c in classes) or "user" in node_id:
            role = "user"
        # 如果文本很长或含代码、表格、Markdown 特征，倾向 chatgpt
        elif len(text) > 200 or "```" in inner_html or "<code" in inner_html or "<pre" in inner_html or "<table" in inner_html:
            role = "chatgpt"
        # 含 "assistant:" 前缀
        elif text.strip().lower().startswith("assistant:") or text.strip().lower().startswith("chatgpt:"):
            role = "chatgpt"
        # 含 "user:" 或以问句结尾倾向 user（短）
        elif text.strip().endswith("?") and len(text) < 200:
            role = "user"
        else:
            # 若 node 在 DOM 中包含子节点里明显是回答/段落（heuristic）
            # 保守处理：若节点下包含多段 <p> 或多行换行，倾向 chatgpt
            if inner_html.count("<p") >= 2 or inner_html.count("<br") >= 3 or inner_html.count("\n") >= 4:
                role = "chatgpt"
            else:
                role = "user"

        # Push message (保留原始 inner_html)
        messages.append({"role": role, "html": inner_html, "text": text})

    # 最后尝试合并相邻同 role 的短块，避免碎片化
    merged = []
    for m in messages:
        if merged and merged[-1]["role"] == m["role"]:
            # 合并 HTML 和 text（保留之间换行）
            merged[-1]["html"] += "<hr style='display:none;margin:0;padding:0;border:0;height:0;'>\n" + m["html"]
            merged[-1]["text"] += "\n\n" + m["text"]
        else:
            merged.append(m)
    return merged

# ---------------------------
# 生成交互式网页模板
# ---------------------------
def generate_output_html(messages, title="ChatGPT 会话 复原 v4", generated_time=None):
    if generated_time is None:
        generated_time = datetime.utcnow().isoformat() + "Z"
    # Header + core css/js（包含搜索、折叠、主题切换、代码高亮、MathJax）
    header = f"""<!DOCTYPE html>
<html lang="zh">
<head>
<meta charset="utf-8">
<meta name="viewport" content="width=device-width,initial-scale=1">
<title>{html.escape(title)}</title>

<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.8.0/styles/github.min.css">
<script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.8.0/highlight.min.js"></script>
<script>window.addEventListener('load', ()=>{{hljs.highlightAll();}});</script>

<!-- MathJax -->
<script src="https://polyfill.io/v3/polyfill.min.js?features=es6"></script>
<script id="MathJax-script" async
  src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
</script>

<style>
:root {{
  --bg: #f7f7f8;
  --user-bg: #e6e6e6;
  --assistant-bg: #ffffff;
  --assistant-accent: #0b93f6;
  --text: #0b0b0b;
  --muted: #555;
}}
html,body{{height:100%;}}
body {{
  margin:0;
  font-family: Inter, "Segoe UI", Roboto, "Helvetica Neue", Arial, sans-serif;
  background: var(--bg);
  color: var(--text);
  padding: 20px;
}}
.container {{
  max-width:1100px;
  margin:0 auto;
}}
.header-row {{
  display:flex;align-items:center;justify-content:space-between;margin-bottom:14px;
}}
.toolbar {{
  display:flex;gap:8px;align-items:center;
}}
.btn {{
  padding:8px 10px;border-radius:8px;border:1px solid rgba(0,0,0,0.08);background:#fff;cursor:pointer;
}}
.search-input{{padding:8px;border-radius:8px;border:1px solid #ddd;min-width:220px;}}

.chat-area {{ background:transparent;padding:8px 0; }}

.message {{
  display:block;
  clear:both;
  padding:14px 18px;
  margin:10px 0;
  border-radius:12px;
  max-width:78%;
  word-wrap:break-word;
  box-shadow: 0 6px 18px rgba(16,24,40,0.04);
}}
.user {{
  background:var(--user-bg);
  float:left;
}}
.chatgpt {{
  background:var(--assistant-bg);
  float:right;
  border-left:4px solid var(--assistant-accent);
}}

.meta {{ font-size:12px;color:var(--muted);margin-bottom:6px; }}

.message pre {{
  background:#f0f0f0;
  padding:12px;border-radius:8px;overflow:auto;cursor:pointer;
  max-height:220px;
}}
.message table{{ border-collapse: collapse; width:100%;}}
.message table, .message th, .message td {{ border:1px solid #ccc; padding:6px;}}
.message img{{ max-width:100%; display:block; margin:8px auto; border-radius:6px; }}

.clearfix{{clear:both;}}

/* night theme */
body.night {{
  --bg: #0b1020;
  --user-bg: #151826;
  --assistant-bg: #071127;
  --assistant-accent: #2fa6ff;
  --text: #e6eef6;
  --muted: #9fb0c8;
}}
.header-info {{ font-size:12px;color:var(--muted); }}
.controls-row {{ display:flex; gap:8px; align-items:center; }}
.copy-btn {{ margin-left:8px; padding:4px 8px; font-size:12px; }}

.search-highlight {{ background: rgba(255,230,100,0.6); padding:0 3px; border-radius:2px; }}
</style>
</head>
<body>
<div class="container">
  <div class="header-row">
    <div>
      <h2 style="margin:0 0 6px 0;">{html.escape(title)}</h2>
      <div class="header-info">生成时间: {generated_time} · 文件渲染自导出 HTML</div>
    </div>
    <div class="toolbar">
      <input class="search-input" id="globalSearch" placeholder="搜索会话（回车高亮）" />
      <button class="btn" id="btnSearch">搜索</button>
      <button class="btn" id="btnClearSearch">清除</button>
      <button class="btn" id="btnCollapseAll">折叠全部</button>
      <button class="btn" id="btnExpandAll">展开全部</button>
      <button class="btn" id="btnToggleTheme">夜间/白天</button>
      <a class="btn" id="downloadAll" href="#" download="chatgpt_session_rendered.html">下载当前页面</a>
    </div>
  </div>

  <div class="chat-area" id="chatArea">
"""
    # Messages insertion
    body_parts = []
    idx = 0
    for m in messages:
        idx += 1
        role = m.get("role","user")
        css_role = "chatgpt" if role=="chatgpt" else ("user" if role=="user" else "user")
        # Sanitize minimal but keep HTML inside (we assume the user trusts these files)
        inner_html = m.get("html","")
        # ensure images lazy-load
        inner_html = inner_html.replace("<img ", '<img loading="lazy" ')
        # wrap with meta and message container
        html_msg = f'''
    <div class="message {css_role}" data-index="{idx}">
      <div class="meta">{html.escape(role)} · 消息 #{idx}</div>
      <div class="content">{inner_html}</div>
      <div style="margin-top:8px;">
        <button class="btn copy-btn" onclick="copyMessage({idx})">复制文字</button>
        <button class="btn copy-btn" onclick="copyHTML({idx})">复制 HTML</button>
      </div>
    </div>
    <div class="clearfix"></div>
'''
        body_parts.append(html_msg)

    footer = """
  </div> <!-- chat-area -->
</div> <!-- container -->

<script>
function qs(sel){return document.querySelector(sel);}
function qsa(sel){return Array.from(document.querySelectorAll(sel));}

// copy helpers
function copyToClipboard(text){
  navigator.clipboard.writeText(text).then(()=>{ alert('已复制到剪贴板'); },(e)=>{ alert('复制失败：'+e); });
}
function copyMessage(idx){
  const el = document.querySelector('.message[data-index="'+idx+'"] .content');
  copyToClipboard(el.innerText.trim());
}
function copyHTML(idx){
  const el = document.querySelector('.message[data-index="'+idx+'"] .content');
  copyToClipboard(el.innerHTML.trim());
}

// collapse/expand behavior for pre and tables and large images
function setUpCollapsibles(){
  qsa('.message pre').forEach(pre=>{
    pre.style.maxHeight = '220px';
    pre.style.overflow = 'hidden';
    pre.dataset.collapsed = '1';
    pre.addEventListener('click', ()=>{
      if(pre.dataset.collapsed==='1'){ pre.style.maxHeight='none'; pre.dataset.collapsed='0'; }
      else { pre.style.maxHeight='220px'; pre.dataset.collapsed='1'; }
    });
  });
  qsa('.message table').forEach(tbl=>{
    const wrapper = document.createElement('div');
    wrapper.style.maxHeight = '220px';
    wrapper.style.overflow = 'hidden';
    wrapper.style.cursor = 'pointer';
    tbl.parentNode.insertBefore(wrapper, tbl);
    wrapper.appendChild(tbl);
    wrapper.dataset.collapsed = '1';
    wrapper.addEventListener('click', ()=>{
      if(wrapper.dataset.collapsed==='1'){ wrapper.style.maxHeight='none'; wrapper.dataset.collapsed='0'; }
      else { wrapper.style.maxHeight='220px'; wrapper.dataset.collapsed='1'; }
    });
  });
  qsa('.message img').forEach(img=>{
    // large images: set max-height and allow click to toggle
    img.style.maxHeight = '360px';
    img.style.objectFit = 'contain';
    img.style.cursor = 'pointer';
    img.dataset.collapsed = '1';
    img.addEventListener('click', ()=>{
      if(img.dataset.collapsed==='1'){ img.style.maxHeight='none'; img.dataset.collapsed='0'; }
      else { img.style.maxHeight='360px'; img.dataset.collapsed='1'; }
    });
  });
}

// global collapse/expand
function collapseAll(){
  qsa('.message pre').forEach(pre=>{ pre.style.maxHeight='220px'; pre.dataset.collapsed='1'; });
  qsa('.message table').forEach(wrapper=>{ if(wrapper.parentNode && wrapper.parentNode.querySelector('div')){ /* skip */ }});
  qsa('.message img').forEach(img=>{ img.style.maxHeight='360px'; img.dataset.collapsed='1'; });
}
function expandAll(){
  qsa('.message pre').forEach(pre=>{ pre.style.maxHeight='none'; pre.dataset.collapsed='0'; });
  qsa('.message table').forEach(tbl=>{ if(tbl.parentNode) tbl.parentNode.style.maxHeight='none'; });
  qsa('.message img').forEach(img=>{ img.style.maxHeight='none'; img.dataset.collapsed='0'; });
}

// search
function clearHighlights(){
  qsa('.search-highlight').forEach(el=>{
    const parent = el.parentNode;
    parent.replaceChild(document.createTextNode(el.textContent), el);
  });
}
function highlightSearch(term){
  if(!term) return;
  clearHighlights();
  const re = new RegExp(term.replace(/[-/\\^$*+?.()|[\\]{}]/g, '\\\\$&'), 'ig');
  qsa('.message .content').forEach(node=>{
    // walk text nodes and replace matches with span
    function walk(n){
      if(n.nodeType === 3){
        const match = n.nodeValue.match(re);
        if(match){
          const span = document.createElement('span');
          span.innerHTML = n.nodeValue.replace(re, function(m){ return '<span class="search-highlight">'+m+'</span>'; });
          n.parentNode.replaceChild(span, n);
        }
      } else {
        for(let i=0;i<n.childNodes.length;i++){ walk(n.childNodes[i]); }
      }
    }
    walk(node);
  });
}

// download current page as file
document.getElementById('downloadAll').addEventListener('click', function(e){
  const blob = new Blob([document.documentElement.outerHTML], {type:'text/html'});
  const url = URL.createObjectURL(blob);
  this.href = url;
  setTimeout(()=>URL.revokeObjectURL(url), 10000);
});

// toolbar actions
document.getElementById('btnCollapseAll').addEventListener('click', collapseAll);
document.getElementById('btnExpandAll').addEventListener('click', expandAll);
document.getElementById('btnToggleTheme').addEventListener('click', ()=>{
  document.body.classList.toggle('night');
});
document.getElementById('btnSearch').addEventListener('click', ()=>{
  const q = document.getElementById('globalSearch').value.trim();
  clearHighlights();
  if(q) highlightSearch(q);
});
document.getElementById('btnClearSearch').addEventListener('click', ()=>{
  document.getElementById('globalSearch').value='';
  clearHighlights();
});
document.getElementById('globalSearch').addEventListener('keydown', function(e){
  if(e.key === 'Enter'){ document.getElementById('btnSearch').click(); }
});

// init
setUpCollapsibles();
</script>
</body>
</html>
"""
    return header + "\n".join(body_parts) + footer

# ---------------------------
# 处理文件夹中所有 .html 文件
# ---------------------------
def process_folder(input_dir, output_dir):
    input_dir = Path(input_dir)
    output_dir = Path(output_dir)
    files = sorted([p for p in input_dir.iterdir() if p.suffix.lower() == ".html"])
    if not files:
        print("未找到任何 .html 文件，检查 INPUT_FOLDER 路径是否正确：", input_dir)
        return
    print(f"发现 {len(files)} 个 HTML 文件，开始处理...")
    for p in files:
        try:
            print("处理：", p.name)
            txt, enc = read_file_with_encoding_detection(str(p))
            msgs = extract_messages_from_html(txt)
            out_html = generate_output_html(msgs, title=f"Rendered - {p.name}", generated_time=datetime.utcnow().strftime("%Y-%m-%d %H:%M:%SZ"))
            out_name = p.stem + "_rendered_v4.html"
            out_path = output_dir / out_name
            with open(out_path, "w", encoding="utf-8") as fw:
                fw.write(out_html)
            print("已生成：", out_path)
        except Exception as e:
            print("处理失败：", p.name, e)

# ---------------------------
# 运行
# ---------------------------
process_folder(INPUT_FOLDER, OUTPUT_FOLDER)
print("全部完成。生成文件位于：", OUTPUT_FOLDER)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
发现 1 个 HTML 文件，开始处理...
处理： 解释下文意思.html


/tmp/ipython-input-3456665724.py:445: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  out_html = generate_output_html(msgs, title=f"Rendered - {p.name}", generated_time=datetime.utcnow().strftime("%Y-%m-%d %H:%M:%SZ"))


已生成： /content/drive/MyDrive/chatgpt_html_rendered_v4/解释下文意思_rendered_v4.html
全部完成。生成文件位于： /content/drive/MyDrive/chatgpt_html_rendered_v4


In [ ]:
import os
import chardet
from bs4 import BeautifulSoup
from google.colab import files

# ============= 配置区 =============
# Google Drive 挂载后的文件夹路径
input_folder = "/content/drive/MyDrive/chatgpt_exports"
# 输出文件名
output_file = "chatgpt_only_fullscreen.html"
# =================================

# 收集所有 ChatGPT 消息
chatgpt_messages = []

# 遍历文件夹中的所有文件
for filename in os.listdir(input_folder):
    file_path = os.path.join(input_folder, filename)

    if not os.path.isfile(file_path):
        continue

    # 自动检测编码
    with open(file_path, "rb") as f:
        raw_data = f.read()
    result = chardet.detect(raw_data)
    encoding = result['encoding'] or 'utf-8'

    # 解码并用 BeautifulSoup 解析
    text = raw_data.decode(encoding, errors="ignore")
    soup = BeautifulSoup(text, "html.parser")

    # 遍历所有标签，筛选 ChatGPT 回复
    for tag in soup.find_all(True):
        tag_text = tag.get_text(strip=True)
        if not tag_text:
            continue

        # 用户输入一般很短，排除掉
        if len(tag_text) < 30:
            continue

        # ChatGPT 回复通常较长，或者 class 里含有 chatgpt/assistant
        tag_classes = tag.get("class", [])
        if any("assistant" in c.lower() or "chatgpt" in c.lower() for c in tag_classes) or len(tag_text) > 80:
            chatgpt_messages.append(str(tag))

# 去重，保持顺序
seen = set()
unique_messages = []
for msg in chatgpt_messages:
    if msg not in seen:
        unique_messages.append(msg)
        seen.add(msg)

# 生成 HTML 页面（全屏显示 ChatGPT 回复）
html_template = f"""
<!DOCTYPE html>
<html lang="zh">
<head>
  <meta charset="UTF-8">
  <title>ChatGPT Only</title>
  <style>
    body {{
      font-family: Arial, sans-serif;
      margin: 0;
      padding: 20px;
      background: #f9f9f9;
    }}
    .chatgpt-message {{
      width: 100%;
      background: #ffffff;
      border-radius: 10px;
      padding: 20px;
      margin: 15px 0;
      box-shadow: 0 2px 6px rgba(0,0,0,0.1);
      white-space: pre-wrap;
    }}
  </style>
</head>
<body>
  {"".join(f'<div class="chatgpt-message">{msg}</div>' for msg in unique_messages)}
</body>
</html>
"""

# 保存到文件
with open(output_file, "w", encoding="utf-8") as f:
    f.write(html_template)

# 提供下载
files.download(output_file)
print(f"✅ 已生成并打包 ChatGPT 回复，文件名: {output_file}")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ 已生成并打包 ChatGPT 回复，文件名: chatgpt_only_fullscreen.html


In [ ]:
import os
import chardet
from bs4 import BeautifulSoup
from google.colab import files

# ============= 配置区 =============
input_folder = "/content/drive/MyDrive/chatgpt_exports"
output_file = "chatgpt_only_clean.html"
# =================================

chatgpt_messages = []

for filename in os.listdir(input_folder):
    file_path = os.path.join(input_folder, filename)

    if not os.path.isfile(file_path):
        continue

    with open(file_path, "rb") as f:
        raw_data = f.read()
    result = chardet.detect(raw_data)
    encoding = result['encoding'] or 'utf-8'
    text = raw_data.decode(encoding, errors="ignore")
    soup = BeautifulSoup(text, "html.parser")

    for tag in soup.find_all(True):
        tag_text = tag.get_text(strip=True)
        if not tag_text:
            continue
        if len(tag_text) < 30:  # 短的，跳过（用户输入）
            continue

        tag_classes = tag.get("class", [])
        if any("assistant" in c.lower() or "chatgpt" in c.lower() for c in tag_classes) or len(tag_text) > 80:
            # 去掉图片和 script
            for img in tag.find_all("img"):
                img.decompose()
            for script in tag.find_all("script"):
                script.decompose()
            for style in tag.find_all("style"):
                style.decompose()

            # 只保留纯净 HTML 结构
            chatgpt_messages.append(tag.get_text())

# 去重
seen = set()
unique_messages = []
for msg in chatgpt_messages:
    if msg not in seen:
        unique_messages.append(msg)
        seen.add(msg)

# 生成精简 HTML
html_template = f"""
<!DOCTYPE html>
<html lang="zh">
<head>
  <meta charset="UTF-8">
  <title>ChatGPT Only Clean</title>
  <style>
    body {{
      font-family: Arial, sans-serif;
      margin: 0;
      padding: 20px;
      background: #f9f9f9;
    }}
    .chatgpt-message {{
      width: 100%;
      background: #ffffff;
      border-radius: 10px;
      padding: 20px;
      margin: 15px 0;
      box-shadow: 0 2px 6px rgba(0,0,0,0.1);
      white-space: pre-wrap;
    }}
  </style>
</head>
<body>
  {"".join(f'<div class="chatgpt-message">{msg}</div>' for msg in unique_messages)}
</body>
</html>
"""

with open(output_file, "w", encoding="utf-8") as f:
    f.write(html_template)

files.download(output_file)
print(f"✅ 已生成精简版 ChatGPT 回复，文件名: {output_file}")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ 已生成精简版 ChatGPT 回复，文件名: chatgpt_only_clean.html


In [ ]:
# ---------------------------
# 安装依赖（Colab 环境）
# ---------------------------
!pip install -q chardet beautifulsoup4 markdown pygments

# ---------------------------
# 导入与配置
# ---------------------------
import os
import chardet
from bs4 import BeautifulSoup
import html
from pathlib import Path
from datetime import datetime
from google.colab import drive, files

# 修改这里
INPUT_FOLDER = "/content/drive/MyDrive/chatgpt_exports"   # 输入文件夹
OUTPUT_FOLDER = "/content/drive/MyDrive/chatgpt_html_rendered_clean"  # 输出文件夹

# ---------------------------
# 挂载 Google Drive
# ---------------------------
drive.mount('/content/drive', force_remount=False)
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# ---------------------------
# 工具函数：安全读取并检测编码
# ---------------------------
def read_file_with_encoding_detection(path):
    with open(path, "rb") as f:
        raw = f.read()
    res = chardet.detect(raw)
    enc = res.get("encoding") or "utf-8"
    try:
        text = raw.decode(enc, errors="ignore")
    except Exception:
        text = raw.decode("utf-8", errors="ignore")
    return text, enc

# ---------------------------
# 提取消息：只保留 ChatGPT
# ---------------------------
def extract_chatgpt_messages(html_text):
    soup = BeautifulSoup(html_text, "html.parser")
    messages = []

    for tag in soup.find_all(True):
        text = tag.get_text(separator="\n", strip=True)
        if not text:
            continue

        classes = [c.lower() for c in (tag.get("class") or [])]
        node_id = (tag.get("id") or "").lower()
        inner_html = tag.decode_contents()

        # 判断是否 ChatGPT
        is_chatgpt = False
        if any("assistant" in c or "chatgpt" in c for c in classes):
            is_chatgpt = True
        elif "assistant" in node_id:
            is_chatgpt = True
        elif len(text) > 200 or "```" in inner_html or "<pre" in inner_html:
            is_chatgpt = True

        if is_chatgpt:
            messages.append({"html": inner_html, "text": text})

    return messages

# ---------------------------
# 生成 HTML（ChatGPT 全屏显示）
# ---------------------------
def generate_output_html(messages, title="ChatGPT 会话（只保留回答，全屏）"):
    header = f"""<!DOCTYPE html>
<html lang="zh">
<head>
<meta charset="utf-8">
<title>{html.escape(title)}</title>
<style>
body {{
  font-family: Arial, sans-serif;
  margin: 0;
  padding: 20px;
  background: #f9f9f9;
  color: #222;
}}
.chatgpt-message {{
  width: 100%;   /* ✅ 全屏宽度 */
  background: #ffffff;
  border-radius: 10px;
  padding: 20px;
  margin: 20px 0;
  box-shadow: 0 2px 6px rgba(0,0,0,0.1);
  line-height: 1.6;
  word-wrap: break-word;
}}
.chatgpt-message pre {{
  background: #f0f0f0;
  padding: 10px;
  border-radius: 6px;
  overflow-x: auto;
}}
.chatgpt-message table {{
  border-collapse: collapse;
  width: 100%;
}}
.chatgpt-message table, .chatgpt-message th, .chatgpt-message td {{
  border: 1px solid #ccc;
  padding: 6px;
}}
.chatgpt-message img {{
  max-width: 100%;
  display: block;
  margin: 10px auto;
}}
</style>
</head>
<body>
<h2>{html.escape(title)}</h2>
"""

    footer = "</body></html>"

    body_html = ""
    for m in messages:
        body_html += f'<div class="chatgpt-message">{m["html"]}</div>\n'

    return header + body_html + footer

# ---------------------------
# 主逻辑：处理文件夹内所有导出文件
# ---------------------------
for filename in os.listdir(INPUT_FOLDER):
    if not filename.endswith(".html"):
        continue
    file_path = os.path.join(INPUT_FOLDER, filename)
    text, enc = read_file_with_encoding_detection(file_path)
    msgs = extract_chatgpt_messages(text)
    out_html = generate_output_html(msgs, title=f"{filename} - ChatGPT 回答")

    out_path = os.path.join(OUTPUT_FOLDER, filename.replace(".html", "_clean.html"))
    with open(out_path, "w", encoding="utf-8") as f:
        f.write(out_html)
    print(f"✅ 已处理: {filename} -> {out_path}")

# ---------------------------
# 打包并下载
# ---------------------------
import shutil
shutil.make_archive("/content/chatgpt_only_clean", 'zip', OUTPUT_FOLDER)
files.download("/content/chatgpt_only_clean.zip")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ 已处理: 解释下文意思.html -> /content/drive/MyDrive/chatgpt_html_rendered_clean/解释下文意思_clean.html


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:

# ---------------------------
# 精简且稳定的 Colab 脚本：只保留 ChatGPT 回复（全屏），去除垃圾与内联图片
# 直接复制到 Google Colab 执行
# ---------------------------

# 安装依赖（Colab）
!pip install -q chardet beautifulsoup4

# ---------------------------
# 导入
# ---------------------------
import os, re, shutil
import chardet
from bs4 import BeautifulSoup, Comment
import html
from pathlib import Path
from datetime import datetime
from google.colab import drive, files

# ---------------------------
# 配置（请按需修改）
# ---------------------------
INPUT_FOLDER = "/content/drive/MyDrive/chatgpt_exports"   # <- 修改为你 Drive 的输入文件夹
OUTPUT_FOLDER = "/content/chatgpt_html_clean_outputs"     # 输出到此临时目录（会被打包并下载）
REMOVE_IMAGES = True   # 是否删除所有图片（True 可显著减小体积）
MIN_ASSISTANT_LENGTH = 120  # 判定为 assistant 的最小纯文本长度阈值（可调整）

# ---------------------------
# 挂载 Google Drive
# ---------------------------
drive.mount('/content/drive', force_remount=False)
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# ---------------------------
# 读取并检测编码（并修剪开头垃圾）
# ---------------------------
def read_file_with_encoding_detection(path):
    with open(path, "rb") as f:
        raw = f.read()
    res = chardet.detect(raw)
    enc = res.get("encoding") or "utf-8"
    try:
        text = raw.decode(enc, errors="ignore")
    except Exception:
        text = raw.decode("utf-8", errors="ignore")
    # **去掉开头的非 HTML 元数据 / 垃圾**：从第一个 '<' 开始
    first_lt = text.find('<')
    if 0 <= first_lt < 2000:
        text = text[first_lt:]
    # 也去掉文件末尾可能的奇怪控制字符
    text = text.strip('\x00\x01\x02\x03\x04\x05\x06\x07\x08\x0b\x0c\x0e\x0f')
    return text, enc

# ---------------------------
# HTML 片段安全清洗：只保留允许的标签和少量属性
# ---------------------------
def sanitize_fragment(fragment_html, remove_images=True):
    allowed_tags = {
        'p':[], 'br':[], 'pre':[], 'code':[], 'blockquote':[],
        'ul':[], 'ol':[], 'li':[], 'strong':[], 'b':[], 'em':[], 'i':[],
        'a':['href','title','target','rel'], 'table':[], 'thead':[], 'tbody':[],
        'tr':[], 'th':[], 'td':[]
    }
    soup = BeautifulSoup(fragment_html, "html.parser")
    # remove scripts, styles, comments
    for s in soup(['script','style']):
        s.decompose()
    for c in soup.find_all(string=lambda text:isinstance(text, Comment)):
        c.extract()
    # images: remove base64 or all images
    for img in soup.find_all('img'):
        src = (img.get('src') or '').strip()
        if remove_images or src.lower().startswith('data:'):
            img.decompose()
        else:
            # keep image but lazy-load and drop inline attrs
            img_attrs = {}
            if src:
                img_attrs['src'] = src
            if img.get('alt'):
                img_attrs['alt'] = img.get('alt')
            img_attrs['loading'] = 'lazy'
            img.attrs = img_attrs
    # sanitize tags: unwrap disallowed tags, keep allowed tags and allowed attrs only
    for tag in list(soup.find_all(True)):
        if tag.name not in allowed_tags:
            tag.unwrap()
        else:
            # keep only allowed attributes (and sanitize href)
            keep = allowed_tags.get(tag.name, [])
            attrs = dict(tag.attrs)
            for k in list(attrs.keys()):
                if k not in keep:
                    del tag.attrs[k]
            if tag.name == 'a' and tag.has_attr('href'):
                href = tag['href'].strip()
                # remove javascript: or data: hrefs
                if href.lower().startswith('javascript:') or href.lower().startswith('data:'):
                    del tag.attrs['href']
                else:
                    # keep absolute/relative links
                    tag.attrs['href'] = href
                    tag.attrs['target'] = '_blank'
                    tag.attrs['rel'] = 'noopener noreferrer'
    # collapse excessive whitespace
    cleaned = str(soup)
    cleaned = re.sub(r'\n{3,}', '\n\n', cleaned)
    return cleaned

# ---------------------------
# 更严格地只提取 ChatGPT / assistant 段落
# ---------------------------
def extract_chatgpt_messages(html_text):
    soup = BeautifulSoup(html_text, "html.parser")
    # 预先去掉 scripts/styles/comments
    for s in soup(['script','style']):
        s.decompose()
    for c in soup.find_all(string=lambda text:isinstance(text, Comment)):
        c.extract()
    messages = []
    # Candidate nodes: 优先查找带 class/id 与 message 相关的节点，但若找不到再遍历所有 block-level 标签
    # 寻找明显包含 assistant 的节点
    candidates = []
    # 先找类名/id里显式含 assistant/chatgpt 的节点
    for tag in soup.find_all(True):
        classes = [c.lower() for c in (tag.get('class') or [])]
        tid = (tag.get('id') or '').lower()
        if any('assistant' in c or 'chatgpt' in c for c in classes) or 'assistant' in tid or 'chatgpt' in tid:
            candidates.append(tag)
    # 如果没有显式标识，就找 message-like 的容器（class 包含 message/msg/row）
    if not candidates:
        for tag in soup.find_all(True):
            classes = [c.lower() for c in (tag.get('class') or [])]
            if any(x in ' '.join(classes) for x in ['message','msg','row','bubble']):
                candidates.append(tag)
    # 如果还为空，退回到 body 下的直接块级子节点
    if not candidates:
        body = soup.body or soup
        for tag in body.find_all(recursive=False):
            candidates.append(tag)

    # 对候选节点逐一判定是否为 assistant
    for node in candidates:
        text = node.get_text(separator="\n", strip=True)
        if not text or len(text.strip())==0:
            continue
        # 检查祖先是否标记为 user（若是则跳过）
        is_user_ancestor = False
        anc = node
        while anc.parent and anc.parent != soup:
            anc = anc.parent
            anc_classes = [c.lower() for c in (anc.get('class') or [])]
            anc_id = (anc.get('id') or '').lower()
            if any('user' in c or 'from-user' in c or 'you' in c for c in anc_classes) or 'user' in anc_id:
                is_user_ancestor = True
                break
        if is_user_ancestor:
            continue
        classes = [c.lower() for c in (node.get('class') or [])]
        node_id = (node.get('id') or '').lower()
        inner_html = node.decode_contents()
        # 判定规则（严格，只保留概率高的 assistant 段）
        is_assistant = False
        # 明确标识
        if any('assistant' in c or 'chatgpt' in c or 'ChatGPT' in c for c in classes) or 'assistant' in node_id or 'ChatGPT' in node_id. or 'chatgpt' in node_id:
            is_assistant = True
        # 含有代码块、表格或 markdown code fence
        elif node.find('pre') or node.find('code') or node.find('table') or '```' in inner_html:
            is_assistant = True
        # 文本足够长且不是典型问题（以 ? 结束且很短）
        elif len(text) >= MIN_ASSISTANT_LENGTH and not (text.strip().endswith('?') and len(text) < 200):
            is_assistant = True
        # 明确以 "assistant:" / "chatgpt:" 前缀开始
        elif text.strip().lower().startswith('assistant:') or text.strip().lower().startswith('chatgpt:'):
            is_assistant = True
        # 否则跳过（避免把用户短句 / 标题 /菜单等当作 assistant）
        if not is_assistant:
            continue
        # sanitize fragment (去除 base64 图等，保留 code/table/links)
        cleaned_html = sanitize_fragment(inner_html, remove_images=REMOVE_IMAGES)
        # after sanitize, skip if nothing left
        if not cleaned_html or len(BeautifulSoup(cleaned_html,"html.parser").get_text(strip=True)) < 20:
            continue
        messages.append({"html": cleaned_html, "text": node.get_text(separator="\n",strip=True)})
    # 合并相邻相同来源（按出现顺序）
    merged = []
    for m in messages:
        if merged and merged[-1].get('html') and merged[-1].get('text') and True:
            # 合并文本和 html：插入分隔以保持段落
            merged[-1]['html'] += "\n\n" + m['html']
            merged[-1]['text'] += "\n\n" + m['text']
        else:
            merged.append(m)
    # 去重并保留顺序（按 text）
    seen = set()
    unique = []
    for m in merged:
        t = m['text'].strip()
        if t in seen:
            continue
        seen.add(t)
        unique.append(m)
    return unique

# ---------------------------
# 生成简洁的输出 HTML（ChatGPT 回复全屏显示）
# ---------------------------
def generate_output_html(messages, title="ChatGPT Replies (clean)"):
    header = f"""<!DOCTYPE html>
<html lang="zh">
<head>
<meta charset="utf-8">
<meta name="viewport" content="width=device-width,initial-scale=1">
<title>{html.escape(title)}</title>
<style>
body {{ font-family: Arial, sans-serif; margin:0; padding:20px; background:#f7f7f7; color:#111; }}
.container {{ max-width:1200px; margin:0 auto; }}
.header {{ margin-bottom:12px; }}
.reply {{ width:100%; background:#fff; border-radius:10px; padding:18px; margin:14px 0; box-shadow:0 2px 8px rgba(0,0,0,0.06); line-height:1.6; word-wrap:break-word; }}
.reply pre {{ background:#f4f4f4; padding:10px; border-radius:6px; overflow:auto; }}
.reply table {{ border-collapse:collapse; width:100%; }}
.reply table, .reply th, .reply td {{ border:1px solid #ddd; padding:6px; }}
</style>
</head>
<body>
<div class="container">
  <div class="header"><h2>{html.escape(title)}</h2><div style="color:#666;font-size:13px">生成时间: {datetime.utcnow().strftime("%Y-%m-%d %H:%M:%SZ")}</div></div>
"""
    body = ""
    if not messages:
        body += '<div style="color:#666">未识别到 ChatGPT 回复（请检查输入文件或调整 MIN_ASSISTANT_LENGTH）。</div>\n'
    else:
        for m in messages:
            body += '<div class="reply">\n'
            body += m['html']
            body += '\n</div>\n'
    footer = """</div></body></html>"""
    return header + body + footer

# ---------------------------
# 批量处理目录中的 HTML 文件
# ---------------------------
input_dir = Path(INPUT_FOLDER)
output_dir = Path(OUTPUT_FOLDER)
output_dir.mkdir(parents=True, exist_ok=True)

files_processed = 0
for p in sorted(input_dir.glob("*.html")):
    try:
        print("处理：", p.name)
        txt, enc = read_file_with_encoding_detection(str(p))
        msgs = extract_chatgpt_messages(txt)
        out_html = generate_output_html(msgs, title=f"Cleaned - {p.name}")
        out_name = p.stem + "_clean.html"
        out_path = output_dir / out_name
        with open(out_path, "w", encoding="utf-8") as fw:
            fw.write(out_html)
        print("已生成：", out_path, "| messages:", len(msgs))
        files_processed += 1
    except Exception as e:
        print("处理失败：", p.name, e)

if files_processed == 0:
    print("没有处理到任何 .html 文件，检查 INPUT_FOLDER 路径是否正确。")

# ---------------------------
# 打包并触发下载（zip）
# ---------------------------
zip_base = "/content/chatgpt_clean_outputs"
shutil.make_archive(zip_base, 'zip', str(output_dir))
zip_path = zip_base + ".zip"
print("生成 zip：", zip_path, "（开始下载）")
files.download(zip_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
处理： AWS_SAA_C03_4.html


/tmp/ipython-input-3546487727.py:227: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  <div class="header"><h2>{html.escape(title)}</h2><div style="color:#666;font-size:13px">生成时间: {datetime.utcnow().strftime("%Y-%m-%d %H:%M:%SZ")}</div></div>


已生成： /content/chatgpt_html_clean_outputs/AWS_SAA_C03_4_clean.html | messages: 1
处理： 解释下文意思.html
已生成： /content/chatgpt_html_clean_outputs/解释下文意思_clean.html | messages: 1
生成 zip： /content/chatgpt_clean_outputs.zip （开始下载）


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# ---------------------------
# 精简且稳定的 Colab 脚本：只保留 ChatGPT 回复（全屏），去除「您说：」及垃圾内容
# 直接复制到 Google Colab 执行
# ---------------------------

# 安装依赖（Colab）
!pip install -q chardet beautifulsoup4

# ---------------------------
# 导入
# ---------------------------
import os, re, shutil
import chardet
from bs4 import BeautifulSoup, Comment
import html
from pathlib import Path
from datetime import datetime
from google.colab import drive, files

# ---------------------------
# 配置（请按需修改）
# ---------------------------
INPUT_FOLDER = "/content/drive/MyDrive/chatgpt_exports"   # <- 修改为你 Drive 的输入文件夹
OUTPUT_FOLDER = "/content/chatgpt_html_clean_outputs"     # 输出到此临时目录（会被打包并下载）
REMOVE_IMAGES = True   # 是否删除所有图片（True 可显著减小体积）
MIN_ASSISTANT_LENGTH = 20  # 判定为 assistant 的最小纯文本长度阈值

# ---------------------------
# 挂载 Google Drive
# ---------------------------
drive.mount('/content/drive', force_remount=False)
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# ---------------------------
# 读取并检测编码（并修剪开头垃圾）
# ---------------------------
def read_file_with_encoding_detection(path):
    with open(path, "rb") as f:
        raw = f.read()
    res = chardet.detect(raw)
    enc = res.get("encoding") or "utf-8"
    try:
        text = raw.decode(enc, errors="ignore")
    except Exception:
        text = raw.decode("utf-8", errors="ignore")
    # 去掉开头的非 HTML 元数据 / 垃圾
    first_lt = text.find('<')
    if 0 <= first_lt < 2000:
        text = text[first_lt:]
    # 去掉末尾可能的控制字符
    text = text.strip('\x00\x01\x02\x03\x04\x05\x06\x07\x08\x0b\x0c\x0e\x0f')
    return text, enc

# ---------------------------
# HTML 片段安全清洗
# ---------------------------
def sanitize_fragment(fragment_html, remove_images=True):
    allowed_tags = {
        'p':[], 'br':[], 'pre':[], 'code':[], 'blockquote':[],
        'ul':[], 'ol':[], 'li':[], 'strong':[], 'b':[], 'em':[], 'i':[],
        'a':['href','title','target','rel'], 'table':[], 'thead':[], 'tbody':[],
        'tr':[], 'th':[], 'td':[]
    }
    soup = BeautifulSoup(fragment_html, "html.parser")
    for s in soup(['script','style']):
        s.decompose()
    for c in soup.find_all(string=lambda text:isinstance(text, Comment)):
        c.extract()
    for img in soup.find_all('img'):
        src = (img.get('src') or '').strip()
        if remove_images or src.lower().startswith('data:'):
            img.decompose()
        else:
            img_attrs = {}
            if src:
                img_attrs['src'] = src
            if img.get('alt'):
                img_attrs['alt'] = img.get('alt')
            img_attrs['loading'] = 'lazy'
            img.attrs = img_attrs
    for tag in list(soup.find_all(True)):
        if tag.name not in allowed_tags:
            tag.unwrap()
        else:
            keep = allowed_tags.get(tag.name, [])
            attrs = dict(tag.attrs)
            for k in list(attrs.keys()):
                if k not in keep:
                    del tag.attrs[k]
            if tag.name == 'a' and tag.has_attr('href'):
                href = tag['href'].strip()
                if href.lower().startswith('javascript:') or href.lower().startswith('data:'):
                    del tag.attrs['href']
                else:
                    tag.attrs['href'] = href
                    tag.attrs['target'] = '_blank'
                    tag.attrs['rel'] = 'noopener noreferrer'
    cleaned = str(soup)
    cleaned = re.sub(r'\n{3,}', '\n\n', cleaned)
    return cleaned

# ---------------------------
# 提取 ChatGPT 段落（只保留 "ChatGPT 说：" 内容）
# ---------------------------
def extract_chatgpt_messages(html_text):
    soup = BeautifulSoup(html_text, "html.parser")
    for s in soup(['script','style']):
        s.decompose()
    for c in soup.find_all(string=lambda text:isinstance(text, Comment)):
        c.extract()
    messages = []
    for node in soup.find_all(True):
        text = node.get_text(separator="\n", strip=True)
        if not text:
            continue
        # 去掉用户输入
        if text.startswith("您说："):
            continue
        # 只保留 ChatGPT 回复
        if text.startswith("ChatGPT 说："):
            text_clean = text.replace("ChatGPT 说：", "", 1).strip()
            if len(text_clean) < MIN_ASSISTANT_LENGTH:
                continue
            inner_html = node.decode_contents()
            cleaned_html = sanitize_fragment(inner_html, remove_images=REMOVE_IMAGES)
            messages.append({"html": cleaned_html, "text": text_clean})
    return messages

# ---------------------------
# 生成输出 HTML
# ---------------------------
def generate_output_html(messages, title="ChatGPT Replies (clean)"):
    header = f"""<!DOCTYPE html>
<html lang="zh">
<head>
<meta charset="utf-8">
<meta name="viewport" content="width=device-width,initial-scale=1">
<title>{html.escape(title)}</title>
<style>
body {{ font-family: Arial, sans-serif; margin:0; padding:20px; background:#f7f7f7; color:#111; }}
.container {{ max-width:1200px; margin:0 auto; }}
.header {{ margin-bottom:12px; }}
.reply {{ width:100%; background:#fff; border-radius:10px; padding:18px; margin:14px 0; box-shadow:0 2px 8px rgba(0,0,0,0.06); line-height:1.6; word-wrap:break-word; }}
.reply pre {{ background:#f4f4f4; padding:10px; border-radius:6px; overflow:auto; }}
.reply table {{ border-collapse:collapse; width:100%; }}
.reply table, .reply th, .reply td {{ border:1px solid #ddd; padding:6px; }}
</style>
</head>
<body>
<div class="container">
  <div class="header"><h2>{html.escape(title)}</h2><div style="color:#666;font-size:13px">生成时间: {datetime.utcnow().strftime("%Y-%m-%d %H:%M:%SZ")}</div></div>
"""
    body = ""
    if not messages:
        body += '<div style="color:#666">未识别到 ChatGPT 回复。</div>\n'
    else:
        for m in messages:
            body += '<div class="reply">\n'
            body += m['html']
            body += '\n</div>\n'
    footer = "</div></body></html>"
    return header + body + footer

# ---------------------------
# 批量处理目录中的 HTML 文件
# ---------------------------
input_dir = Path(INPUT_FOLDER)
output_dir = Path(OUTPUT_FOLDER)
output_dir.mkdir(parents=True, exist_ok=True)

files_processed = 0
for p in sorted(input_dir.glob("*.html")):
    try:
        print("处理：", p.name)
        txt, enc = read_file_with_encoding_detection(str(p))
        msgs = extract_chatgpt_messages(txt)
        out_html = generate_output_html(msgs, title=f"Cleaned - {p.name}")
        out_name = p.stem + "_clean.html"
        out_path = output_dir / out_name
        with open(out_path, "w", encoding="utf-8") as fw:
            fw.write(out_html)
        print("已生成：", out_path, "| messages:", len(msgs))
        files_processed += 1
    except Exception as e:
        print("处理失败：", p.name, e)

if files_processed == 0:
    print("没有处理到任何 .html 文件，检查 INPUT_FOLDER 路径是否正确。")

# ---------------------------
# 打包并触发下载（zip）
# ---------------------------
zip_base = "/content/chatgpt_clean_outputs"
shutil.make_archive(zip_base, 'zip', str(output_dir))
zip_path = zip_base + ".zip"
print("生成 zip：", zip_path, "（开始下载）")
files.download(zip_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
处理： AWS_SAA_C03_4.html


/tmp/ipython-input-1436145591.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  <div class="header"><h2>{html.escape(title)}</h2><div style="color:#666;font-size:13px">生成时间: {datetime.utcnow().strftime("%Y-%m-%d %H:%M:%SZ")}</div></div>


已生成： /content/chatgpt_html_clean_outputs/AWS_SAA_C03_4_clean.html | messages: 0
处理： 解释下文意思.html
已生成： /content/chatgpt_html_clean_outputs/解释下文意思_clean.html | messages: 386
生成 zip： /content/chatgpt_clean_outputs.zip （开始下载）


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# ---------------------------
# 精简且稳定的 Colab 脚本：支持中/日/英三语言界面
# ---------------------------

!pip install -q chardet beautifulsoup4

import os, re, shutil
import chardet
from bs4 import BeautifulSoup, Comment
import html
from pathlib import Path
from datetime import datetime
from google.colab import drive, files

# ---------------------------
# 配置
# ---------------------------
INPUT_FOLDER = "/content/drive/MyDrive/chatgpt_exports"
OUTPUT_FOLDER = "/content/chatgpt_html_clean_outputs"
REMOVE_IMAGES = True
MIN_ASSISTANT_LENGTH = 20

drive.mount('/content/drive', force_remount=False)
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# ---------------------------
# 读取文件并检测编码
# ---------------------------
def read_file_with_encoding_detection(path):
    with open(path, "rb") as f:
        raw = f.read()
    res = chardet.detect(raw)
    enc = res.get("encoding") or "utf-8"
    try:
        text = raw.decode(enc, errors="ignore")
    except Exception:
        text = raw.decode("utf-8", errors="ignore")
    first_lt = text.find('<')
    if 0 <= first_lt < 2000:
        text = text[first_lt:]
    text = text.strip('\x00\x01\x02\x03\x04\x05\x06\x07\x08\x0b\x0c\x0e\x0f')
    return text, enc

# ---------------------------
# HTML 片段安全清洗
# ---------------------------
def sanitize_fragment(fragment_html, remove_images=True):
    allowed_tags = {
        'p':[], 'br':[], 'pre':[], 'code':[], 'blockquote':[],
        'ul':[], 'ol':[], 'li':[], 'strong':[], 'b':[], 'em':[], 'i':[],
        'a':['href','title','target','rel'], 'table':[], 'thead':[], 'tbody':[],
        'tr':[], 'th':[], 'td':[]
    }
    soup = BeautifulSoup(fragment_html, "html.parser")
    for s in soup(['script','style']):
        s.decompose()
    for c in soup.find_all(string=lambda text:isinstance(text, Comment)):
        c.extract()
    for img in soup.find_all('img'):
        src = (img.get('src') or '').strip()
        if remove_images or src.lower().startswith('data:'):
            img.decompose()
        else:
            img_attrs = {}
            if src:
                img_attrs['src'] = src
            if img.get('alt'):
                img_attrs['alt'] = img.get('alt')
            img_attrs['loading'] = 'lazy'
            img.attrs = img_attrs
    for tag in list(soup.find_all(True)):
        if tag.name not in allowed_tags:
            tag.unwrap()
        else:
            keep = allowed_tags.get(tag.name, [])
            attrs = dict(tag.attrs)
            for k in list(attrs.keys()):
                if k not in keep:
                    del tag.attrs[k]
            if tag.name == 'a' and tag.has_attr('href'):
                href = tag['href'].strip()
                if href.lower().startswith('javascript:') or href.lower().startswith('data:'):
                    del tag.attrs['href']
                else:
                    tag.attrs['href'] = href
                    tag.attrs['target'] = '_blank'
                    tag.attrs['rel'] = 'noopener noreferrer'
    cleaned = str(soup)
    cleaned = re.sub(r'\n{3,}', '\n\n', cleaned)
    return cleaned

# ---------------------------
# 提取 ChatGPT 段落（支持中/日/英）
# ---------------------------
def extract_chatgpt_messages(html_text):
    soup = BeautifulSoup(html_text, "html.parser")
    for s in soup(['script','style']):
        s.decompose()
    for c in soup.find_all(string=lambda text:isinstance(text, Comment)):
        c.extract()

    messages = []
    # 多语言匹配前缀
    USER_PREFIXES = ["您说：", "あなた:", "You:"]
    GPT_PREFIXES = ["ChatGPT 说：", "ChatGPT:", "ChatGPT :"]

    for node in soup.find_all(True):
        text = node.get_text(separator="\n", strip=True)
        if not text:
            continue
        # 跳过用户输入（任一语言）
        if any(text.startswith(prefix) for prefix in USER_PREFIXES):
            continue
        # 仅保留 ChatGPT 回复（任一语言）
        for prefix in GPT_PREFIXES:
            if text.startswith(prefix):
                text_clean = text.replace(prefix, "", 1).strip()
                if len(text_clean) < MIN_ASSISTANT_LENGTH:
                    continue
                inner_html = node.decode_contents()
                cleaned_html = sanitize_fragment(inner_html, remove_images=REMOVE_IMAGES)
                messages.append({"html": cleaned_html, "text": text_clean})
                break  # 找到匹配即退出循环
    return messages

# ---------------------------
# 生成输出 HTML
# ---------------------------
def generate_output_html(messages, title="ChatGPT Replies (clean)"):
    header = f"""<!DOCTYPE html>
<html lang="zh">
<head>
<meta charset="utf-8">
<meta name="viewport" content="width=device-width,initial-scale=1">
<title>{html.escape(title)}</title>
<style>
body {{ font-family: Arial, sans-serif; margin:0; padding:20px; background:#f7f7f7; color:#111; }}
.container {{ max-width:1200px; margin:0 auto; }}
.header {{ margin-bottom:12px; }}
.reply {{ width:100%; background:#fff; border-radius:10px; padding:18px; margin:14px 0; box-shadow:0 2px 8px rgba(0,0,0,0.06); line-height:1.6; word-wrap:break-word; }}
.reply pre {{ background:#f4f4f4; padding:10px; border-radius:6px; overflow:auto; }}
.reply table {{ border-collapse:collapse; width:100%; }}
.reply table, .reply th, .reply td {{ border:1px solid #ddd; padding:6px; }}
</style>
</head>
<body>
<div class="container">
  <div class="header"><h2>{html.escape(title)}</h2><div style="color:#666;font-size:13px">生成时间: {datetime.utcnow().strftime("%Y-%m-%d %H:%M:%SZ")}</div></div>
"""
    body = ""
    if not messages:
        body += '<div style="color:#666">未识别到 ChatGPT 回复。</div>\n'
    else:
        for m in messages:
            body += '<div class="reply">\n'
            body += m['html']
            body += '\n</div>\n'
    footer = "</div></body></html>"
    return header + body + footer

# ---------------------------
# 批量处理目录中的 HTML 文件
# ---------------------------
input_dir = Path(INPUT_FOLDER)
output_dir = Path(OUTPUT_FOLDER)
output_dir.mkdir(parents=True, exist_ok=True)

files_processed = 0
for p in sorted(input_dir.glob("*.html")):
    try:
        print("处理：", p.name)
        txt, enc = read_file_with_encoding_detection(str(p))
        msgs = extract_chatgpt_messages(txt)
        out_html = generate_output_html(msgs, title=f"Cleaned - {p.name}")
        out_name = p.stem + "_clean.html"
        out_path = output_dir / out_name
        with open(out_path, "w", encoding="utf-8") as fw:
            fw.write(out_html)
        print("已生成：", out_path, "| messages:", len(msgs))
        files_processed += 1
    except Exception as e:
        print("处理失败：", p.name, e)

if files_processed == 0:
    print("没有处理到任何 .html 文件，检查 INPUT_FOLDER 路径是否正确。")

# ---------------------------
# 打包并下载 zip
# ---------------------------
zip_base = "/content/chatgpt_clean_outputs"
shutil.make_archive(zip_base, 'zip', str(output_dir))
zip_path = zip_base + ".zip"
print("生成 zip：", zip_path, "（开始下载）")
files.download(zip_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
处理： AWS_SAA_C03_4.html


/tmp/ipython-input-758364400.py:148: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  <div class="header"><h2>{html.escape(title)}</h2><div style="color:#666;font-size:13px">生成时间: {datetime.utcnow().strftime("%Y-%m-%d %H:%M:%SZ")}</div></div>


已生成： /content/chatgpt_html_clean_outputs/AWS_SAA_C03_4_clean.html | messages: 78
处理： 解释下文意思.html
已生成： /content/chatgpt_html_clean_outputs/解释下文意思_clean.html | messages: 386
生成 zip： /content/chatgpt_clean_outputs.zip （开始下载）


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>